# Preparação

## Anotações e lembretes

Na parte de timetracking eu não pego o horário exato de quando a atividade foi iniciada ou finalizada. Isso porque esse dado fica bem complicado de puxar e tratar, principalmente quando as pessoas fazem a atividade "picada" - começa e para mais de uma vez até finalizar. Como isso não é uma informação relevante hoje, optei por não puxar / tratar

boards_execucao e conversao_mes tão com colunas float vindo com ponto em vez de vírgula. averiguar e consertar

## Imports

In [1]:
# imports gerais
import requests
import json
import pprint
import pandas as pd
import datetime as dt
import numpy as np
import time

# imports pra validação da API do Sheets
import os
import google.oauth2.credentials
import google_auth_oauthlib.flow
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

## Variáveis Universais

In [2]:
HOJE = pd.Timestamp.today().strftime("%Y-%m-%d")

## Autorizações

In [3]:
#!ls /datasets/gdrive-pessoal/Projetos/consultorias/interno/

client_secret_161456402816-df34cah61b789bbsbfksq8q6to6c009b.apps.googleusercontent.com.json
openai_key.env
sheets_api_auth.json
teste_dados_brutos.json
test.txt


In [4]:
# Autorização Monday

monday_apiKey = "" # inserir chave
monday_apiUrl = "https://api.monday.com/v2"
headers = {
    "Authorization" : monday_apiKey,
    "API-Version": '2023-10'
}

In [5]:
# Autorização Sheets

SECRET_FILE = f"" # inserir filepath do .json
CREDS_PATH = f"" # inserir filepath do .json

with open(CREDS_PATH, 'r') as token:
    credentials = json.load(token)
    creds = google.oauth2.credentials.Credentials(**credentials)

### Reautorizar Sheets

In [6]:
# Gerar link pra pegar o código de autorização

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
    SECRET_FILE,
    SCOPES)


def gerar_link_auth():
    flow.redirect_uri = '' # inserir URL de autorização

    authorization_url, state = flow.authorization_url(
        access_type='offline',
        include_granted_scopes='true')

    print(authorization_url)

    return None

In [7]:
# Copiar a URL gerada pelo redirecionamento e colar em "authorization_response"

def reautorizar_sheets(auth_url):
    authorization_response = auth_url
    flow.fetch_token(authorization_response=authorization_response)
    credentials = flow.credentials

    temp_creds = {'token': credentials.token,
        'refresh_token': credentials.refresh_token,
        'token_uri': credentials.token_uri,
        'client_id': credentials.client_id,
        'client_secret': credentials.client_secret,
        'scopes': credentials.scopes}

    with open(CREDS_PATH, 'w') as creds_file:
        json.dump(temp_creds, creds_file)

    print('salvo')

    return None

In [8]:
#gerar_link_auth()

In [9]:
url_autorizacao = "" # inserir URL recebida na célula acima

#reautorizar_sheets(url_autorizacao)

## Funções de Extração

### Funções Auxiliares à Extração

In [10]:
# Transforma a lista de colunas que vão ser puxadas em string, em vez de list

def gerar_lista_colunas(dict_colunas):
    lista_colunas = '['

    for coluna in dict_colunas:
        lista_colunas += ('"' + coluna + '", ')

    return (lista_colunas[:-2] + ']')

In [11]:
def gerar_lista_grupos(dict_grupos):
    lista_grupos = []
    for grupo in dict_grupos[0]['groups']:
        lista_grupos.append(grupo['id'])

    return lista_grupos

### Metadados

In [12]:
# Extrai os ID's de todos os workspaces

def extrair_metadados_workspaces():
    workspaces_list_query = '''query { 
        workspaces ( limit: 50 )  { 
            id
            name
        }
    }'''

    metadados_workspaces = (
        json.loads(
            requests.get(
                url=monday_apiUrl, 
                json={'query' : workspaces_list_query}, 
                headers=headers)
            .text)['data']['workspaces']
    )

    return pd.DataFrame.from_dict(metadados_workspaces)

In [13]:
def extrair_metadados_boards():
    query_boards = '''query { 
        boards ( limit: 700 ) { 
            id 
            name
            board_folder_id
            workspace_id
            items_count
        } 
    }'''

    metadados_boards = (
        json.loads(
            requests.get(
                url=monday_apiUrl, 
                json={'query' : query_boards}, 
                headers=headers)
            .text)['data']['boards']
    )

    return pd.DataFrame.from_dict(metadados_boards)

In [14]:
# extrai os metadados da coluna, principalmente pra puxar o ID #[3607290028]

def extrair_metadados_colunas(board_id):
    query_colunas = '''query { 
        boards ( ids: %s ) { 
            id 
            name 
            items_page {
                items { 
                    id 
                    name 
                    group { title }
                    column_values { 
                        column {
                            id
                            title
                        }
                        id
                        type 
                        value 
                        text 
                    }
                } 
            }
        } 
    }''' % str(board_id)

    metadados_colunas = (
        json.loads(
            requests.get(
                url=monday_apiUrl, 
                json={'query' : query_colunas}, 
                headers=headers)
            .text)['data']['boards'][0]['items_page']['items'][0]['column_values']
    )

    lista_cols = []
    for col in metadados_colunas:
        dict_col = {
            'id': col['id'],
            'title': col['column']['title'],
            'type': col['type']
        }
        lista_cols.append(dict_col)

    return pd.DataFrame.from_dict(lista_cols)

### Dados board

In [15]:
def extrair_dados_grupos(board_id):
    query_grupos = '''query { 
        boards ( ids: %s ) { 
            groups {
                title
                id
            }
        } 
    }''' % str(board_id)

    dados_grupos_board = (
        json.loads(
            requests.get(
                url=monday_apiUrl, 
                json={'query' : query_grupos}, 
                headers=headers)
            .text)['data']['boards']
    )

    return dados_grupos_board

In [16]:
def extrair_dados_board_v3(board_id, group_id, column_ids, print_an=False):
    agora =(pd.Timestamp.today() - pd.Timedelta(hours=3)).strftime("%Y-%m-%d %r")

    #print(f"Começando agora: {agora}")

    query_dados_board = '''query { 
        complexity {
            before
            query
            after
            reset_in_x_seconds
        }
        boards ( ids: %s ) { 
            id
            name
            items_count 
            items_page ( limit: 500, query_params: {rules: [{column_id: "group", compare_value: ["%s"]}] } ) {
                items { 
                    id
                    name
                    group { 
                        title
                        id
                    }
                    parent_item { id }
                    column_values (ids: %s ) { 
                        id
                        type 
                        value 
                        text 
                        ... on StatusValue {
                            is_done
                        }
                        ... on MirrorValue {
                            display_value 
                        }
                        ... on BoardRelationValue {
                            display_value
                        }
                        ... on TimeTrackingValue {
                            history {
                                created_at
                                started_user_id
                                started_at
                                ended_at
                            }
                        }
                    }
                    subitems {
                        id
                        name
                        group { 
                            title
                            id
                        }
                        parent_item {
                            id
                        }
                        column_values {
                            id
                            type
                            value
                            text
                        }
                    }
                }
            }
        }
    }'''  % (board_id, group_id, column_ids)

    dados_extraidos_board = (
        json.loads(
            requests.get(
                url=monday_apiUrl, 
                json={'query' : query_dados_board}, 
                headers=headers)
            .text)
    )

    controle_board = '''
    nome_board = dados_extraidos_board['data']['boards'][0]['name']
    itens_board = dados_extraidos_board['data']['boards'][0]['items_page']['items']

    print(f"\nExtração finalizada.\nBoard: {nome_board}\n")
    if len(itens_board) == 0:
        print(f"Grupo vazio. ID: {group_id}\n")
    else:
        nome_grupo = itens_board[0]['group']['title']
        print(f"Grupo: {nome_grupo}\n")
    
    print("-----")
    '''

    dados_complexidade = dados_extraidos_board['data']['complexity']
    complexidade_antes = int(dados_complexidade['before'])
    complexidade_query = int(dados_complexidade['query'])
    complexidade_depois = int(dados_complexidade['after'])
    complexidade_reset = int(dados_complexidade['reset_in_x_seconds'])
    queries_restantes = int(round((complexidade_depois / complexidade_query), 0))

    #print("\nDADOS DA COMPLEXIDADE\n")
    #print(f"Antes: {complexidade_antes}\nQuery: {complexidade_query}")
    #print(f"Depois: {complexidade_depois}\nReset em: {complexidade_reset}")
    if print_an == True:
        print(f"Queries restantes no ritmo atual: {queries_restantes}\nReset em: {complexidade_reset}\n")
        print("------------------------------------------------")

    return dados_extraidos_board

### Loop Extração

In [17]:
def loop_extracao_grupos_v2(board_id, dict_colunas, print_an=True, ctrl_ext=False):
    dados_itens_board = []

    grupos_board = gerar_lista_grupos(extrair_dados_grupos(board_id))
    colunas_board = gerar_lista_colunas(dict_colunas)

    agora = (pd.Timestamp.today() - pd.Timedelta(hours=3)).strftime("%Y-%m-%d %r")

    if print_an == True:
        print(f"Começando extração de {len(grupos_board)} grupos.\n")

    # extrair os dados, grupo por grupo
    contador = 0
    for grupo in grupos_board:
        contador += 1

        if print_an == True:
            print(f"Extraindo grupo {contador}/{len(grupos_board)}")
        
        resposta = (
            extrair_dados_board_v3(
                board_id, # board
                grupo, # grupo
                colunas_board, # colunas
                ctrl_ext
            )
        )

        complexi = resposta['data']['complexity']
        complexidade_antes = int(complexi['before'])
        complexidade_query = int(complexi['query'])
        complexidade_depois = int(complexi['after'])
        complexidade_reset = int(complexi['reset_in_x_seconds'])
        queries_restantes = int(round((complexidade_depois / complexidade_query), 0))

        if len(grupos_board) >= queries_restantes:
            time.sleep(complexidade_reset + 3)

        for item in resposta['data']['boards']:#[0]['items_page']['items']:
            if len(resposta['data']['boards'][0]['items_page']['items']) > 0:
                dados_itens_board.append(item)
    
    if print_an == True:
        print("\nFinalizado.")

    return dados_itens_board

### Subitems pra itens

In [18]:
def subitems_into_items(dados_brutos):
    dict_subitems_board = [{
        'id': dados_brutos[0]['id'],
        'name': dados_brutos[0]['name'],
        'items_page': {'items': []}
    }]

    for dados_grupo in dados_brutos:
        for item in dados_grupo['items_page']['items']:
            for subitem in item['subitems']:
                dict_subitems_board[0]['items_page']['items'].append(subitem)
    
    return dict_subitems_board

## Funções de Tratamento

### Json pra Dict

In [19]:
def board_json_pra_dict(dados_board, granularidade, print_item=False):
    dados_achatados_board = []
    time_alocado = []
    #timetracking = []

    imprimir = print_item

    for board in dados_board:
        board_id = board['id']
        board_name = board['name']

        for item in board['items_page']['items']:
            if imprimir == True:
                print(item)

            item_dict = {
                    'data_extracao': HOJE,
                    'id': item['id'],
                    'id_quadro': board_id,
                    'quadro_origem': board_name,
                    'item_name': item['name'],
                    'item_group_title': item['group']['title'] #grupo_quadro
                }

            if item['parent_item'] != None:
                item_dict['id_elemento_dono'] = item['parent_item']['id']
            
            for column_value in item['column_values']:
                column_id = column_value['id']

                if column_value['type'] == 'formula':
                    item_dict[column_id] = None
                
                elif column_value['type'] in ['status', 'numbers', 'date', 'text', 'long_text', 'rating', 'checkbox']:
                    if column_value['text'] != None:
                        item_dict[column_id] = column_value['text']
                
                elif column_value['type'] in ['mirror']:
                    if column_value['display_value'] != None:
                        item_dict[column_id] = column_value['display_value']

                elif column_value['type'] == 'board_relation':
                    if column_value['display_value'] == '':
                        item_dict[column_id] = ''
                    else:
                        item_dict[column_id] = str(json.loads(column_value['value'])['linkedPulseIds'][0]['linkedPulseId'])
            
                elif column_value['type'] == 'timeline':
                    if column_value['value'] != None:
                        timerange_values = json.loads(column_value['value'])
                        item_dict[f'{column_id}_from'] = timerange_values['from']
                        item_dict[f'{column_id}_to'] = timerange_values['to']

                elif column_value['type'] == 'people':
                    if column_value['value'] != None:
                        person_list = json.loads(column_value['value'])['personsAndTeams']
                        for person in person_list:
                            allocated_team_dict = {
                                'id_item': item['id'],
                                'quadro_origem': board_name,
                                'granularidade': granularidade,
                                'id_pessoa': person['id']
                            }
                            time_alocado.append(allocated_team_dict)

                elif column_value['type'] == 'time_tracking':
                    if column_value['value'] != None:
                        column_value_dict = json.loads(column_value['value'])
                        item_dict['duracao_total_segundos'] = column_value_dict['duration']

            dados_achatados_board.append(item_dict)

    return {
        'dados_achatados_board': dados_achatados_board,
        'time_alocado': time_alocado
    }

### Colunas dos DFs

In [20]:
def objeto_pra_float(df, colunas):
    for coluna in colunas:
        df[coluna] = (
            df[coluna]
            .replace(r'^\s*$', np.nan, regex=True)
            .astype('float64')
            #.fillna(0)
        )
    
    return None

In [21]:
def objeto_pra_datetime(df, colunas):
    for coluna in colunas:
        df[coluna] = df[coluna].str.split().str[0].astype('datetime64[ns]')
    
    return None

In [22]:
def datetime_pra_date(df, colunas):
    for coluna in colunas:
        df[coluna] = (
            df[coluna]
            .dt.date
            .fillna("")
        )
    
    return None

In [23]:
def date_pra_string(df, colunas):
    for coluna in colunas:
        df[coluna] = (
            df[coluna]
            .astype('string')
            .str.replace('.',',')
        )

    return None

In [24]:
def float_pra_string(df, colunas):
    for coluna in colunas:
        df[coluna] = (
            df[coluna]
            .astype('string')
            .str.replace('.',',')
        )
    
    return None

## Outras funções

In [25]:
dict_ref_colunas = {
    1: 'A',
    2: 'B',
    3: 'C',
    4: 'D',
    5: 'E',
    6: 'F',
    7: 'G',
    8: 'H',
    9: 'I',
    10: 'J'
}

## Load de Tabelas Estáticas

In [26]:
GDRIVE_ESTATICAS = r"/datasets/gdrive-pessoal/Projetos/consultorias/Stoic/Estáticas/"

fp_calendario = f"{GDRIVE_ESTATICAS}dim_calendario.csv"

df_calendario = pd.read_csv(filepath_or_buffer=fp_calendario, sep=';')

## Outras Variáveis Úteis

In [27]:
GDRIVE_FOLDER = r""
GDRIVE_BRUTOS = r""
GDRIVE_APINOVA = r""

In [28]:
todos_os_boards = extrair_metadados_boards()
hoje = (pd.Timestamp.today() - pd.Timedelta(hours=3)).strftime("%Y-%m-%d")

KeyError: 'data'

# Extração e Tratamento

## ADs Pessoal

### Extração v2

In [ ]:
#extrair_metadados_colunas(3815412117)

In [ ]:
# Selecionar só as colunas que interessam atualmente e definir nome estático pra cada uma

colunas_ad_pessoal = {
    'status8': 'pessoa_avaliada',
    'color': 'trimestre',
    'numeric': 'transparencia',
    'numeric2': 'excelencia',
    'numeric5': 'disciplina',
    'numeric8': 'responsabilidade',
    'numeric23': 'empatia',
    'numeric0': 'impacto',
    'formula': 'media_ad_pessoal',
    'date5': 'data_avaliacao',
    'color9': 'area',
    'observa__es': 'observacoes', # até aqui são dados de coluna as colunas
    'item_name': 'pessoa_avaliadora', # daqui pra baixo são dados do board e item, não de coluna
    'item_group_title': 'ano_avaliacao', 
}

In [ ]:
dados_brutos_ad_pessoal = loop_extracao_grupos_v2(3815412117, colunas_ad_pessoal)

Começando extração de 3 grupos.

Extraindo grupo 1/3
Extraindo grupo 2/3
Extraindo grupo 3/3

Finalizado.


In [ ]:
dados_achatados_ad_pessoal = (
    board_json_pra_dict(dados_brutos_ad_pessoal, 'elemento_ad_pessoal', False)['dados_achatados_board']
)

### Tratamento

In [ ]:
df_ad_pessoal = (
    pd.DataFrame.from_dict(dados_achatados_ad_pessoal)
    .rename(columns=colunas_ad_pessoal)
    .fillna(np.nan)
)

In [ ]:
# DEFINIR QUAIS COLUNAS PRECISAM SER TRATADAS DE QUE FORMA

colunas_data_ad_pessoal = [
    'data_extracao',
    'data_avaliacao'
    ]

colunas_float_ad_pessoal = [
    'transparencia',
    'excelencia',
    'disciplina',
    'responsabilidade',
    'empatia',
    'impacto',
    'media_ad_pessoal'
    ]


# TRANSFORMAR AS COLUNAS NOS SEUS RESPECTIVOS FORMATOS

#objeto_pra_datetime(df_ad_pessoal, colunas_data_ad_pessoal)
objeto_pra_float(df_ad_pessoal, colunas_float_ad_pessoal)

In [ ]:
df_ad_pessoal['media_ad_pessoal'] = round(
    (df_ad_pessoal['transparencia'] + 
    df_ad_pessoal['excelencia'] + 
    df_ad_pessoal['disciplina'] + 
    df_ad_pessoal['responsabilidade'] + 
    df_ad_pessoal['empatia'] + 
    df_ad_pessoal['impacto']
    ) / 6, 2
)

In [ ]:
# DEIXAR TUDO COMO STRING OU OBJETO PRA SUBIR NO SHEETS

#datetime_pra_date(df_ad_pessoal, colunas_data_ad_pessoal)
float_pra_string(df_ad_pessoal, colunas_float_ad_pessoal)
date_pra_string(df_ad_pessoal, colunas_data_ad_pessoal)

df_ad_pessoal = df_ad_pessoal.fillna('')

In [ ]:
df_ad_pessoal

,data_extracao,id,id_quadro,quadro_origem,pessoa_avaliadora,ano_avaliacao,pessoa_avaliada,trimestre,transparencia,excelencia,disciplina,responsabilidade,empatia,impacto,media_ad_pessoal,data_avaliacao,area,observacoes
0,2024-06-10,6458037226,3815412117,ADs Pessoal,Artur,2024,João Neves,1° Ciclo,"2,25","2,5","2,5","2,75","2,25","2,42","2,44",,M&A,
1,2024-06-10,6516183159,3815412117,ADs Pessoal,Davih Nakamura,2024,Ariana Fernandes,1° Ciclo,"2,25","2,0","1,5","2,5","2,0","1,75","2,0",,Consultoria,
2,2024-06-10,6516200351,3815412117,ADs Pessoal,Davih Nakamura,2024,João Mateus,1° Ciclo,"2,25","2,25","2,5","2,25","2,0","1,75","2,17",,Consultoria,
3,2024-06-10,6629954213,3815412117,ADs Pessoal,Felipe Manara,2024,Rodrigo Paim,1° Ciclo,"2,25","2,0","2,5","2,25","2,5","2,0","2,25",2024-05-13,Comercial,
4,2024-06-10,6636794999,3815412117,ADs Pessoal,Alan,2024,Eduardo Felipe,1° Ciclo,"2,25","2,75","2,5","2,5","2,25","2,5","2,46",2024-05-14,M&A,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2024-06-10,3815498282,3815412117,ADs Pessoal,Davih Nakamura,2022,Davih Nakamura,3° Ciclo,"3,0","2,5","3,0","3,0","3,0","2,0","2,75",2022-12-31,Consultoria,
64,2024-06-10,3815504392,3815412117,ADs Pessoal,João Vítor Neves,2022,João Neves,3° Ciclo,"2,5","2,0","3,0","2,5","2,25","2,5","2,46",2022-12-31,M&A,
65,2024-06-10,3815507791,3815412117,ADs Pessoal,Letícia Ribeiro,2022,Letícia Ribeiro,3° Ciclo,"2,25","1,75","2,5","2,5","3,0","2,5","2,42",2022-12-31,Consultoria,
66,2024-06-10,3815508221,3815412117,ADs Pessoal,Rodrigo Paim,2022,Rodrigo Paim,3° Ciclo,"2,25","3,0","2,25","3,0","3,0","2,75","2,71",2022-12-31,Comercial,


## ADs Projetos

### Extração v2

In [ ]:
#extrair_metadados_colunas(3815594545)

In [ ]:
# Selecionar só as colunas que interessam atualmente e definir nome estático pra cada uma

colunas_ad_projeto = {
    'color3': 'pessoa',
    'color7': 'escopo_projeto',
    'date': 'data_finalizacao',
    'numeric': 'pre_operacao',
    'numeric2': 'entendimento_inicial',
    'numeric5': 'analise_retroativa',
    'numeric8': 'estruturacao_custos',
    'numeric01': 'concepcao_viabilidade',
    'numeric23': 'analise_mercado',
    'numeric0': 'premissas_crescimento',
    'numeric81': 'analise_viabilidade',
    'numeric84': 'projecoes',
    'numeric4': 'entregas_finais',
    'formula': 'media_consultoria',
    'numeric1': 'transparencia',
    'numeric29': 'excelencia',
    'numeric80': 'disciplina',
    'numeric7': 'responsabilidade',
    'numeric43': 'empatia',
    'numeric75': 'impacto',
    'formula8': 'media_advising',
    'text1': 'feedbacks',
    'numeric3': 'tese_investimento',
    'numeric9': 'pitch_deck',
    'numeric82': 'shortlist',
    'numeric11': 'roadshow',
    'formula6': 'media_mna', # até aqui são dados de coluna as colunas
    'item_name': 'projeto_avaliado', # daqui pra baixo são dados do board e item, não de coluna
    'item_group_title': 'ano_avaliacao',
}

In [ ]:
dados_brutos_ad_projeto = loop_extracao_grupos_v2(3815594545, colunas_ad_projeto)

Começando extração de 4 grupos.

Extraindo grupo 1/4
Extraindo grupo 2/4
Extraindo grupo 3/4
Extraindo grupo 4/4

Finalizado.


In [ ]:
dados_achatados_ad_projeto = (
    board_json_pra_dict(dados_brutos_ad_projeto, 'elemento_ad_projeto', False)['dados_achatados_board']
)

### Tratamento

In [ ]:
df_ad_projeto = (
    pd.DataFrame.from_dict(dados_achatados_ad_projeto)
    .rename(columns=colunas_ad_projeto)
    .fillna(np.nan)
)

In [ ]:
#df_ad_projeto

In [ ]:
# DEFINIR ORDEM DAS COLUNAS PRA NÃO QUEBRAR CASO FAÇAM ALGUMA MUDANÇA NA ORDEM NO FRONT

df_ad_projeto = df_ad_projeto[[
    'data_extracao', 
    'id', 
    'id_quadro', 
    'quadro_origem', 
    'projeto_avaliado', 
    'ano_avaliacao', 
    'pessoa', 
    'escopo_projeto', 
    'data_finalizacao',
    'pre_operacao', 
    'entendimento_inicial', 
    'analise_retroativa',
    'estruturacao_custos', 
    'concepcao_viabilidade', 
    'analise_mercado',
    'premissas_crescimento', 
    'analise_viabilidade', 
    'projecoes',
    'entregas_finais', 
    'media_consultoria', 
    'transparencia', 
    'excelencia',
    'disciplina', 
    'responsabilidade', 
    'empatia', 
    'impacto',
    'media_advising', 
    'feedbacks', 
    'tese_investimento', 
    'pitch_deck',
    'shortlist', 
    'roadshow', 
    'media_mna'
]]

In [ ]:
# DEFINIR QUAIS COLUNAS PRECISAM SER TRATADAS DE QUE FORMA

colunas_data_ad_projeto = [
    'data_extracao',
    'data_finalizacao'
    ]

colunas_float_ad_projeto = [
    'pre_operacao',
    'entendimento_inicial',
    'analise_retroativa',
    'estruturacao_custos',
    'concepcao_viabilidade',
    'analise_mercado',
    'premissas_crescimento',
    'analise_viabilidade',
    'projecoes',
    'entregas_finais',
    'media_consultoria',
    'transparencia',
    'excelencia',
    'disciplina',
    'responsabilidade',
    'empatia',
    'impacto',
    'media_advising',
    'tese_investimento',
    'pitch_deck',
    'shortlist',
    'roadshow',
    'media_mna'
    ]


# TRANSFORMAR AS COLUNAS NOS SEUS RESPECTIVOS FORMATOS

#objeto_pra_datetime(df_ad_projeto, colunas_data_ad_projeto)
objeto_pra_float(df_ad_projeto, colunas_float_ad_projeto)

In [ ]:
df_ad_projeto['media_consultoria'] = round(df_ad_projeto.iloc[:, 9:19].mean(axis=1, skipna=True), 3)

df_ad_projeto['media_advising'] = round(df_ad_projeto.iloc[:, 20:26].mean(axis=1, skipna=True), 3)

df_ad_projeto['media_mna'] = round(df_ad_projeto.iloc[:, 28:32].mean(axis=1, skipna=True), 3)

In [ ]:
# DEIXAR TUDO COMO STRING OU OBJETO PRA SUBIR NO SHEETS

#datetime_pra_date(df_ad_projeto, colunas_data_ad_projeto)
float_pra_string(df_ad_projeto, colunas_float_ad_projeto)
date_pra_string(df_ad_projeto, colunas_data_ad_projeto)

df_ad_projeto = df_ad_projeto.fillna('')

In [ ]:
df_ad_projeto

,data_extracao,id,id_quadro,quadro_origem,projeto_avaliado,ano_avaliacao,pessoa,escopo_projeto,data_finalizacao,pre_operacao,...,responsabilidade,empatia,impacto,media_advising,feedbacks,tese_investimento,pitch_deck,shortlist,roadshow,media_mna
0,2024-06-10,6661806181,3815594545,ADs Projetos,HH Gelados,Novas Respostas,Davih,Valuation,2024-05-10,"2,5",...,,,,,.,,,,,
1,2024-06-10,6661894499,3815594545,ADs Projetos,HH Gelados,Novas Respostas,Ariana,Valuation,2024-05-10,"2,0",...,,,,,Planejar muito bem as etapas antes de executá-...,,,,,
2,2024-06-10,6786453448,3815594545,ADs Projetos,307-3 Engemega,Novas Respostas,Artur,M&A,2024-05-20,,...,,,,,organização comercial. estimativa do esforço.,"4,0","4,0","2,0","2,0","3,0"
3,2024-06-10,6786456813,3815594545,ADs Projetos,307-3 Engemega,Novas Respostas,João Neves,M&A,2024-05-20,,...,,,,,Organização de uma lista de investidores mais ...,"3,0",,,"2,0","2,5"
4,2024-06-10,6786528499,3815594545,ADs Projetos,344-1 Behauz,Novas Respostas,Artur,M&A,2024-06-06,,...,,,,,.,"1,5","2,5","2,25","2,25","2,125"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,2024-06-10,3816466337,3815594545,ADs Projetos,Clube da Imprensa,2022,Davih,Valuation,2022-12-06,"2,0",...,,,,,,,,,,
151,2024-06-10,3816489520,3815594545,ADs Projetos,Clube da Imprensa,2022,Torelly,Valuation,2022-12-06,"2,75",...,,,,,,,,,,
152,2024-06-10,3816509939,3815594545,ADs Projetos,Clube da Imprensa,2022,João Mateus,Valuation,2022-12-06,"2,25",...,,,,,,,,,,
153,2024-06-10,3816522069,3815594545,ADs Projetos,Corina + Cruls,2022,Torelly,Valuation,2022-12-15,"2,5",...,,,,,,,,,,


## Funil Comercial

### Extração v2

In [ ]:
#extrair_metadados_colunas(2390437634)

In [ ]:
# Selecionar só as colunas que interessam atualmente e definir nome estático pra cada uma

colunas_funil = {
    'status': 'status_funil',
    'data0': 'prazo',
    'status_19': 'servico',
    'pessoas': 'equipe',
    'status_1': 'primeiro_encontro',
    'status_13': 'assinou_nda',
    'status_138': 'aceitou_proposta',
    'status_12': 'assinou_contrato',
    'n_meros': 'valor_proposta',
    'numeric2': 'dificuldade',
    'texto': 'indicacao',
    'data': 'data_abertura',
    'data5': 'data_primeira_reuniao',
    'data3': 'data_assinatura_nda',
    'data29': 'data_proposta',
    'data2': 'data_fechamento',
    'status_11': 'origem_lead',
    'f_rmula': 'dias_negociacao', # até aqui são dados de coluna as colunas
    'item_name': 'lead', # daqui pra baixo são dados do board e item, não de coluna
    'item_group_title': 'estagio_negociacao'
}

In [ ]:
dados_brutos_funil = loop_extracao_grupos_v2(2390437634, colunas_funil)

Começando extração de 4 grupos.

Extraindo grupo 1/4
Extraindo grupo 2/4
Extraindo grupo 3/4
Extraindo grupo 4/4

Finalizado.


In [ ]:
dados_achatados_funil = (
    board_json_pra_dict(dados_brutos_funil, 'elemento_funil', False)['dados_achatados_board']
)

### Tratamento

In [ ]:
df_funil = (
    pd.DataFrame.from_dict(dados_achatados_funil)
    .rename(columns=colunas_funil)
)

In [ ]:
# DEFINIR ORDEM DAS COLUNAS PRA NÃO QUEBRAR CASO FAÇAM ALGUMA MUDANÇA NA ORDEM NO FRONT

df_funil = df_funil[[
    'data_extracao', 
    'id', 
    'id_quadro', 
    'quadro_origem', 
    'estagio_negociacao',
    'lead',
    'status_funil', 
    'prazo', 
    'servico',
    'primeiro_encontro',
    'assinou_nda', 
    'aceitou_proposta', 
    'assinou_contrato', 
    'valor_proposta', 
    'dificuldade', 
    'indicacao', 
    'data_abertura',
    'data_primeira_reuniao', 
    'data_assinatura_nda', 
    'origem_lead',
    'data_proposta',
    'data_fechamento', 
    'dias_negociacao'
    ]]

In [ ]:
# DEFINIR QUAIS COLUNAS PRECISAM SER TRATADAS DE QUE FORMA

colunas_data_funil = [
    'data_extracao', 
    'prazo',
    'data_abertura',
    'data_primeira_reuniao',
    'data_assinatura_nda',
    'data_proposta',
    'data_fechamento'
    ]

colunas_float_funil = ['valor_proposta']


# TRANSFORMAR AS COLUNAS NOS SEUS RESPECTIVOS FORMATOS

objeto_pra_datetime(df_funil, colunas_data_funil)
objeto_pra_float(df_funil, colunas_float_funil)

In [ ]:
df_funil['dias_negociacao'] = (
    (df_funil['data_fechamento'] - df_funil['data_abertura'])
    .dt.days
    .fillna(-1)
    .astype('int64')
    .astype('string')
)

df_funil['dias_negociacao'] = np.where(df_funil['dias_negociacao'] == '-1', np.nan, df_funil['dias_negociacao'])

In [ ]:
# DEIXAR TUDO COMO STRING OU OBJETO PRA SUBIR NO SHEETS

datetime_pra_date(df_funil, colunas_data_funil)
float_pra_string(df_funil, colunas_float_funil)
date_pra_string(df_funil, colunas_data_funil)

df_funil = df_funil.fillna('')

In [ ]:
#df_funil

## CSAT

### Extração v2

In [ ]:
#extrair_metadados_colunas(5007761246)

In [ ]:
# Selecionar só as colunas que interessam atualmente e definir nome estático pra cada uma

colunas_csat = {
    'texto_curto2': 'cliente_pessoa',
    'avalia__o_2': 'alinhamento_proposta',
    'avalia__o': 'impacto',
    'avalia__o_1': 'equipe',
    'avalia__o_3': 'nps',
    #'f_rmula': 'media',
    'texto_longo2': 'sugestoes',
    'espelho8': 'escopo_projeto',
    'status': 'status_nps',
    'data': 'data_coleta',
    'conectar_quadros': 'id_item_gerenciamento',
    'espelho5': 'data_finalizacao',
    'pessoas7': 'equipe',
    #'f_rmula7': 'tempo_coleta'
    'item_name': 'cliente_empresa', # até aqui são dados de coluna as colunas
    'item_group_title': 'grupo_resposta' # daqui pra baixo são dados do board e item, não de coluna
}

In [ ]:
dados_brutos_csat = loop_extracao_grupos_v2(5007761246, colunas_csat)

Começando extração de 4 grupos.

Extraindo grupo 1/4
Extraindo grupo 2/4
Extraindo grupo 3/4
Extraindo grupo 4/4

Finalizado.


In [ ]:
#dados_brutos_csat

In [ ]:
dict_csat = board_json_pra_dict(dados_brutos_csat, 'elemento_csat')

### Tratamento

In [ ]:
df_csat_bruto = (
    pd.DataFrame.from_dict(dict_csat['dados_achatados_board'])
    .rename(columns=colunas_csat)
)

df_csat = df_csat_bruto.copy()

In [ ]:
#df_csat

In [ ]:
# DEFINIR QUAIS COLUNAS PRECISAM SER TRATADAS DE QUE FORMA

colunas_data_csat = [
    'data_extracao',
    'data_coleta',
    'data_finalizacao'
    ]

colunas_float_csat = [
    'alinhamento_proposta',
    'impacto',
    'equipe',
    'nps'
    ]


# TRANSFORMAR AS COLUNAS NOS SEUS RESPECTIVOS FORMATOS

objeto_pra_datetime(df_csat, colunas_data_csat)
objeto_pra_float(df_csat, colunas_float_csat)

In [ ]:
df_csat['media_csat'] = round(((df_csat['alinhamento_proposta'] + df_csat['impacto'] + df_csat['equipe']) / 3), 2)

df_csat['dias_coleta'] = (
    (df_csat['data_coleta'] - df_csat['data_finalizacao'])
    .dt.days
    .fillna(-1)
    .astype('int64')
    .astype('string')
)
df_csat['dias_coleta'] = np.where(df_csat['dias_coleta'] == '-1', np.nan, df_csat['dias_coleta'])

In [ ]:
# DEFINIR ORDEM DAS COLUNAS PRA NÃO QUEBRAR CASO FAÇAM ALGUMA MUDANÇA NA ORDEM NO FRONT

df_csat = df_csat[[
    'data_extracao',
    'id',
    'id_item_gerenciamento',
    'id_quadro',
    'quadro_origem',
    'grupo_resposta',
    'cliente_empresa',
    'cliente_pessoa',
    'alinhamento_proposta',
    'impacto',
    'equipe',
    'nps',
    'media_csat',
    'sugestoes',
    'escopo_projeto',
    'status_nps',
    'data_coleta',
    'data_finalizacao',
    'dias_coleta'
]]

In [ ]:
# DEIXAR TUDO COMO STRING OU OBJETO PRA SUBIR NO SHEETS

datetime_pra_date(df_csat, colunas_data_csat)
float_pra_string(df_csat, colunas_float_csat)
float_pra_string(df_csat, ['media_csat'])
date_pra_string(df_csat, colunas_data_csat)

df_csat = df_csat.fillna('')

In [ ]:
df_csat

,data_extracao,id,id_item_gerenciamento,id_quadro,quadro_origem,grupo_resposta,cliente_empresa,cliente_pessoa,alinhamento_proposta,impacto,equipe,nps,media_csat,sugestoes,escopo_projeto,status_nps,data_coleta,data_finalizacao,dias_coleta
0,2024-06-10,6204395381,6034157161,5007761246,CSAT,Novas Respostas,Olube,Marcus Sampaio,"5,0","5,0","5,0","5,0","5,0",,Fairness Opinion,Promotor,2024-03-06,2024-03-01,5
1,2024-06-10,6264368068,5493011516,5007761246,CSAT,Novas Respostas,Bambu Brasil,Bruno Sartório,"4,0","5,0","4,0","3,0","4,33",,M&A,Neutro,2024-03-14,2024-02-09,34
2,2024-06-10,6523039694,,5007761246,CSAT,Novas Respostas,Grupo T2M,Matheus Costa,"5,0","5,0","5,0","5,0","5,0","Gostei muito do Joao e do Davih, gostaria de d...",,Promotor,2024-04-25,,
3,2024-06-10,6523492451,,5007761246,CSAT,Novas Respostas,Ouriço Restaurante - GRUPO TMS,Danilo Paraiso,"5,0","5,0","5,0","5,0","5,0",,,Promotor,2024-04-25,,
4,2024-06-10,6524597818,,5007761246,CSAT,Novas Respostas,"Saveur, Ourico, Moca, Mare",Thiago Paraiso,"5,0","5,0","5,0","5,0","5,0",Estamos finalizando ainda os processos. Mas go...,,Promotor,2024-04-25,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2024-06-10,5277035522,4329656470,5007761246,CSAT,Modelo Antigo 2023,Cidadania4U,Rodrigo Calhau,"5,0","5,0","5,0","5,0","5,0","Trabalho bastante técnica, que conseguiu demon...",Valuation,Promotor,2023-07-18,2023-07-25,-7
80,2024-06-10,5277016046,3935572133,5007761246,CSAT,Modelo Antigo 2023,Brasil Center Shopping,Leonardo Santana,"5,0","5,0","5,0","5,0","5,0",Ainda não conseguimos obter resultados do proj...,Viabilidade Econômica-Financeira,Promotor,2023-07-21,2023-07-14,7
81,2024-06-10,5277016381,4619393097,5007761246,CSAT,Modelo Antigo 2023,Fabrik,ALEX GRIEBELER,"5,0","5,0","5,0","5,0","5,0",PRINCIPALMENTE SABER O VALOR DA MINHA EMPRESA ...,Valuation,Promotor,2023-07-25,2023-07-25,0
82,2024-06-10,5277016798,4223105027,5007761246,CSAT,Modelo Antigo 2023,Ultra Academia,Cristiane Lopes,"5,0","5,0","5,0","5,0","5,0",,M&A,Promotor,2023-07-28,,


### Queries pra usar de exemplo no print

In [ ]:
df_5 = (lambda: _deepnote_execute_sql('SELECT\n    id\n    , CAST(nps AS int) AS "satisfacao"\n    , CAST(data_finalizacao AS date) AS "data_finalizacao"\n    , CAST(data_coleta AS date) AS "data_coleta"\nFROM df_csat_bruto\nWHERE data_finalizacao != \'\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('SELECT\n    id\n    , CAST(nps AS int) AS "satisfacao"\n    , CAST(data_finalizacao AS date) AS "data_finalizacao"\n    , CAST(data_coleta AS date) AS "data_coleta"\nFROM df_csat_bruto\nWHERE data_finalizacao != \'\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
df_5

,id,satisfacao,data_finalizacao,data_coleta
0,6204395381,5,2024-03-01,2024-03-06
1,6264368068,3,2024-02-09,2024-03-14
2,5772830843,3,2023-03-10,2024-01-02
3,5772837360,3,2023-05-05,2024-01-02
4,5828617983,5,2024-01-08,2024-01-10
...,...,...,...,...
64,5277015654,5,2023-07-25,2023-07-07
65,5277034423,5,2023-06-23,2023-07-11
66,5277035522,5,2023-07-25,2023-07-18
67,5277016046,5,2023-07-14,2023-07-21


In [ ]:
df_6 = (lambda: _deepnote_execute_sql('SELECT\n    id\n    , CAST(LEFT(nps, 1) AS INT) AS "satisfacao"\n    , CAST(data_finalizacao AS date) AS "data_finalizacao"\n    , CAST(data_coleta AS date) AS "data_coleta"\n    , CAST(dias_coleta AS INT) AS "dias_coleta"\nFROM df_csat\nWHERE\n    data_finalizacao != \'\'\n    AND dias_coleta != \'\'\n    AND id NOT IN (\'5129521612\', \'5772830843\', \'5772837360\', \'5168190700\')', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('SELECT\n    id\n    , CAST(LEFT(nps, 1) AS INT) AS "satisfacao"\n    , CAST(data_finalizacao AS date) AS "data_finalizacao"\n    , CAST(data_coleta AS date) AS "data_coleta"\n    , CAST(dias_coleta AS INT) AS "dias_coleta"\nFROM df_csat\nWHERE\n    data_finalizacao != \'\'\n    AND dias_coleta != \'\'\n    AND id NOT IN (\'5129521612\', \'5772830843\', \'5772837360\', \'5168190700\')', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
df_6

,id,satisfacao,data_finalizacao,data_coleta,dias_coleta
0,6204395381,5,2024-03-01,2024-03-06,5
1,6264368068,3,2024-02-09,2024-03-14,34
2,5828617983,5,2024-01-08,2024-01-10,2
3,6038985722,5,2024-02-19,2024-02-09,-10
4,6077803758,5,2024-02-26,2024-02-16,-10
5,6080162558,4,2024-02-26,2024-02-16,-10
6,5138876980,5,2023-08-25,2023-09-11,17
7,5138925889,5,2023-08-25,2023-09-11,17
8,5138909156,4,2023-09-08,2023-09-11,3
9,5195771950,5,2023-09-17,2023-09-20,3


In [ ]:
df_7 = (lambda: _deepnote_execute_sql('WITH cte1 AS (\n    SELECT\n        id\n        , CAST(LEFT(nps, 1) AS int) AS "nps"\n        , CAST(dias_coleta AS int) AS "dias_coleta"\n        , CAST(data_coleta AS date) AS "data_coleta"\n        , CAST(data_finalizacao AS date) AS "data_finalizacao"\n        , dias_coleta\n    FROM df_csat\n    WHERE \n        data_finalizacao != \'\'\n        AND dias_coleta != \'\'\n        AND id NOT IN (\'5129521612\', \'5772830843\', \'5772837360\', \'5168190700\')\n),\ncte2 AS (\n    SELECT\n        DATE_TRUNC(\'month\', data_coleta) AS "mes_coleta",\n        ROUND(AVG(nps), 1) AS "media_satisfacao"\n    FROM cte1\n    GROUP BY 1 ORDER BY 1\n)\n\nSELECT * FROM cte2', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('WITH cte1 AS (\n    SELECT\n        id\n        , CAST(LEFT(nps, 1) AS int) AS "nps"\n        , CAST(dias_coleta AS int) AS "dias_coleta"\n        , CAST(data_coleta AS date) AS "data_coleta"\n        , CAST(data_finalizacao AS date) AS "data_finalizacao"\n        , dias_coleta\n    FROM df_csat\n    WHERE \n        data_finalizacao != \'\'\n        AND dias_coleta != \'\'\n        AND id NOT IN (\'5129521612\', \'5772830843\', \'5772837360\', \'5168190700\')\n),\ncte2 AS (\n    SELECT\n        DATE_TRUNC(\'month\', data_coleta) AS "mes_coleta",\n        ROUND(AVG(nps), 1) AS "media_satisfacao"\n    FROM cte1\n    GROUP BY 1 ORDER BY 1\n)\n\nSELECT * FROM cte2', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
df_7

,mes_coleta,media_satisfacao
0,2023-01-01,5.0
1,2023-02-01,4.7
2,2023-03-01,4.8
3,2023-04-01,5.0
4,2023-05-01,4.3
5,2023-06-01,5.0
6,2023-07-01,4.8
7,2023-08-01,5.0
8,2023-09-01,4.8
9,2023-10-01,3.5


## Elementos Atas (KR's)

### Extração - Etapa geral

In [ ]:
# importa os metadados de TODAS as colunas dos boards de Elementos das Atas
#extrair_metadados_colunas(4099791281)
#extrair_metadados_colunas(3951025094)

In [ ]:
# Selecionar só as colunas que interessam atualmente e definir nome estático pra cada uma

colunas_elementos_atas = {
    'person': 'equipe',
    'lookup8': 'escopo',
    'color': 'status_kr',
    'color7': 'cronograma',
    'numeric': 'dias_atraso',
    'date': 'data_finalizacao',
    'data': 'mes_kr',
    'board_relation': 'id_item_projeto', # até aqui são dados de coluna as colunas
    'item_name': 'atividade_kr', # daqui pra baixo são dados do board e item, não de coluna
    'item_group_title': 'projeto_dono', 
}

### Extração - Ativos

In [ ]:
dados_brutos_atas_ativos = loop_extracao_grupos_v2(3951025094, colunas_elementos_atas)

Começando extração de 21 grupos.

Extraindo grupo 1/21
Extraindo grupo 2/21
Extraindo grupo 3/21
Extraindo grupo 4/21
Extraindo grupo 5/21
Extraindo grupo 6/21
Extraindo grupo 7/21
Extraindo grupo 8/21
Extraindo grupo 9/21
Extraindo grupo 10/21
Extraindo grupo 11/21
Extraindo grupo 12/21
Extraindo grupo 13/21
Extraindo grupo 14/21
Extraindo grupo 15/21
Extraindo grupo 16/21
Extraindo grupo 17/21
Extraindo grupo 18/21
Extraindo grupo 19/21
Extraindo grupo 20/21
Extraindo grupo 21/21

Finalizado.


In [ ]:
dict_resposta_ata_ativos = board_json_pra_dict(dados_brutos_atas_ativos, "elemento_ata", False)

In [ ]:
fp_dict_resposta_ata_ativos = f"{GDRIVE_BRUTOS}{hoje}_dict_resposta_ata_ativos.json"

with open(fp_dict_resposta_ata_ativos, "w") as outfile: 
    json.dump(dict_resposta_ata_ativos, outfile)

In [ ]:
df_ata_ativos = pd.DataFrame.from_dict(dict_resposta_ata_ativos['dados_achatados_board'])

In [ ]:
df_ata_ativos

,data_extracao,id,id_quadro,quadro_origem,item_name,item_group_title,lookup8,color,color7,numeric,data,date,board_relation
0,2024-06-10,6224880200,3951025094,Ata Start Semanal Stoic,Concluir V1 da aba de custos e premissas de cr...,402-1 Casapark,Crédito,Em andamento,Em dia,,,,6011849423
1,2024-06-10,6183120442,3951025094,Ata Start Semanal Stoic,Validar BP,Ecosys,M&A,Em andamento,Em dia,,2024-03-01,2024-03-30,5891026985
2,2024-06-10,6176360716,3951025094,Ata Start Semanal Stoic,Enviar documentos necessários para Banco,387-2 Lotus 2.0,Crédito,Em andamento,Em dia,,2024-03-01,2024-03-30,6150304825
3,2024-06-10,5991983031,3951025094,Ata Start Semanal Stoic,Finalizar avaliação de capacidade de pagamento,307-3 Engemega,Crédito,Concluído,Em dia,,2024-02-01,2024-03-01,5971942122
4,2024-06-10,5991984625,3951025094,Ata Start Semanal Stoic,Finalizar captação - 1º Tranche,307-3 Engemega,Crédito,Concluído,Em dia,,2024-02-01,2024-03-01,5971942122
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,2024-06-10,5612601639,3951025094,Ata Start Semanal Stoic,Realizar a captação da 1a Tranche,344-1 Behauz,M&A,Não Concluído,Em dia,,2023-12-01,2023-12-30,4787832963
149,2024-06-10,5759392316,3951025094,Ata Start Semanal Stoic,Definir a nova tese de investimento com VGV in...,344-1 Behauz,M&A,Concluído,Em dia,,2024-01-01,2024-01-30,4787832963
150,2024-06-10,5759398650,3951025094,Ata Start Semanal Stoic,Acompanhar quitação de todos os mútuos,344-1 Behauz,M&A,Concluído,Em dia,,2024-01-01,2024-01-30,4787832963
151,2024-06-10,5995012608,3951025094,Ata Start Semanal Stoic,Definir Go-No Go dos Investidores,344-1 Behauz,M&A,Concluído,Em dia,,2024-02-01,2024-03-01,4787832963


### Teste extração finalizados

### Extração - Finalizados

In [ ]:
dados_brutos_atas_finalizados = loop_extracao_grupos_v2(4099791281, colunas_elementos_atas, ctrl_ext=False)

Começando extração de 95 grupos.

Extraindo grupo 1/95
Extraindo grupo 2/95
Extraindo grupo 3/95
Extraindo grupo 4/95
Extraindo grupo 5/95
Extraindo grupo 6/95
Extraindo grupo 7/95
Extraindo grupo 8/95
Extraindo grupo 9/95
Extraindo grupo 10/95
Extraindo grupo 11/95
Extraindo grupo 12/95
Extraindo grupo 13/95
Extraindo grupo 14/95
Extraindo grupo 15/95
Extraindo grupo 16/95
Extraindo grupo 17/95
Extraindo grupo 18/95
Extraindo grupo 19/95
Extraindo grupo 20/95
Extraindo grupo 21/95
Extraindo grupo 22/95
Extraindo grupo 23/95
Extraindo grupo 24/95
Extraindo grupo 25/95
Extraindo grupo 26/95
Extraindo grupo 27/95
Extraindo grupo 28/95
Extraindo grupo 29/95
Extraindo grupo 30/95
Extraindo grupo 31/95
Extraindo grupo 32/95
Extraindo grupo 33/95
Extraindo grupo 34/95
Extraindo grupo 35/95
Extraindo grupo 36/95
Extraindo grupo 37/95
Extraindo grupo 38/95
Extraindo grupo 39/95
Extraindo grupo 40/95
Extraindo grupo 41/95
Extraindo grupo 42/95
Extraindo grupo 43/95
Extraindo grupo 44/95
Extrain

In [ ]:
dict_resposta_ata_finalizados = board_json_pra_dict(dados_brutos_atas_finalizados, "elemento_ata", False)

In [ ]:
fp_dict_resposta_ata_finalizados = f"{GDRIVE_BRUTOS}{hoje}_dict_resposta_ata_finalizados.json"

with open(fp_dict_resposta_ata_finalizados, "w") as outfile:
    json.dump(dict_resposta_ata_finalizados, outfile)

In [ ]:
df_ata_finalizados = pd.DataFrame.from_dict(dict_resposta_ata_finalizados['dados_achatados_board'])

### Tratamento

In [ ]:
lista_dfs_elementos_atas = [df_ata_ativos, df_ata_finalizados]

df_elementos_atas = (
    pd.concat(lista_dfs_elementos_atas)
    .rename(columns=colunas_elementos_atas)
)

In [ ]:
#df_elementos_atas

In [ ]:
# Trata a coluna "Mês do KR" pra puxar o primeiro dia do mês
# Isso porque no Monday as pessoas às vezes marcam algum dia qualquer do mês, mas eu preciso do dia 01

df_elementos_atas['mes_kr'] = (
    df_elementos_atas['mes_kr']
    .replace(r'^\s*$', '2001-01-01', regex=True)
    .astype('datetime64[ns]')
    .dt.to_period('M')
    .apply(lambda r: r.start_time)
    .dt.date
    .astype('string')
)

df_elementos_atas['mes_kr'] = np.where(
    df_elementos_atas['mes_kr'] == "2001-01-01", 
    "", 
    df_elementos_atas['mes_kr']
)

In [ ]:
df_elementos_atas = df_elementos_atas[
    ['id',
    'id_item_projeto',
    'projeto_dono', 
    'atividade_kr', 
    'status_kr',
    'cronograma', 
    'dias_atraso', 
    'mes_kr',
    'data_finalizacao'
    ]
]

In [ ]:
df_elementos_atas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 557 entries, 0 to 403
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                557 non-null    object
 1   id_item_projeto   557 non-null    object
 2   projeto_dono      557 non-null    object
 3   atividade_kr      557 non-null    object
 4   status_kr         557 non-null    object
 5   cronograma        557 non-null    object
 6   dias_atraso       557 non-null    object
 7   mes_kr            557 non-null    object
 8   data_finalizacao  557 non-null    object
dtypes: object(9)
memory usage: 43.5+ KB


## Subelementos Atas (PDA's)

### Extração - Etapa geral

In [ ]:
# Selecionar só as colunas que interessam atualmente e definir nome estático pra cada uma

colunas_subelementos_atas = {
    'pessoas': 'equipe',
    'status': 'status_pda',
    'date0': 'data_planejada',
    'data': 'data_realizada', # board atas Ativos
    'data_realizada': 'data_realizada', # board atas Finalizados
    'item_name': 'atividade_pda', # daqui pra baixo são dados do board e item, não de coluna
    'item_group_title': 'projeto_dono'
}

### Extração - Ativos

In [ ]:
dados_brutos_subitems_atas_ativos = subitems_into_items(dados_brutos_atas_ativos)

In [ ]:
dict_subitems_atas_ativos = board_json_pra_dict(dados_brutos_subitems_atas_ativos, "subelemento_ata", False)

In [ ]:
df_subitems_atas_ativos = (
    pd.DataFrame.from_dict(dict_subitems_atas_ativos['dados_achatados_board'])
    .rename(columns=colunas_subelementos_atas)
)

### Extração - Finalizados

In [ ]:
dados_brutos_subitems_atas_finalizados = subitems_into_items(dados_brutos_atas_finalizados)

In [ ]:
dict_subitems_atas_finalizados = board_json_pra_dict(dados_brutos_subitems_atas_finalizados, "subelemento_ata", False)

In [ ]:
df_subitems_atas_finalizados = (
    pd.DataFrame.from_dict(dict_subitems_atas_finalizados['dados_achatados_board'])
    .rename(columns=colunas_subelementos_atas)
)

### Tratamento

In [ ]:
lista_dfs_subelementos_atas = [df_subitems_atas_ativos, df_subitems_atas_finalizados]

df_subelementos_atas = (
    pd.concat(lista_dfs_subelementos_atas)
    .rename(columns={'quadro_origem': 'quadro_origem_pda'})
)

In [ ]:
df_subelementos_atas = df_subelementos_atas[[
    'id',
    'id_elemento_dono',
    'quadro_origem_pda',
    'atividade_pda',
    'status_pda',
    'data_planejada',
    'data_realizada'
]]

In [ ]:
df_subelementos_atas['data_planejada'] = df_subelementos_atas['data_planejada'].astype('datetime64[ns]').dt.date
df_subelementos_atas['data_realizada'] = df_subelementos_atas['data_realizada'].astype('datetime64[ns]').dt.date

In [ ]:
# Tratamento pra unificar os dados dos KR's com os dos PDA's
# Seria melhor fazer isso pelo próprio Monday, ver com a Ju

df_subelementos_atas['status_pda'] = df_subelementos_atas['status_pda'].replace('Não Feito', 'Não Concluído')
df_subelementos_atas['status_pda'] = df_subelementos_atas['status_pda'].replace('Feito', 'Concluído')
df_subelementos_atas['status_pda'] = df_subelementos_atas['status_pda'].replace(np.nan, 'Vazio')

In [ ]:
df_subelementos_atas['data_planejada'] = (df_subelementos_atas['data_planejada']
    .astype('string')
    .fillna("")
)

df_subelementos_atas['data_realizada'] = (df_subelementos_atas['data_realizada']
    .astype('string')
    .fillna("")
)

In [ ]:
df_subelementos_atas

,id,id_elemento_dono,quadro_origem_pda,atividade_pda,status_pda,data_planejada,data_realizada
0,6229871262,6224880200,Ata Start Semanal Stoic,Agendar/Realizar reunião com Supera e José,Não Concluído,2024-03-15,
1,6229871713,6224880200,Ata Start Semanal Stoic,Compartilhar lista de informações/documentos c...,Concluído,2024-03-15,2024-03-15
2,6229872137,6224880200,Ata Start Semanal Stoic,Compartilhar cronograma do BP,Não Concluído,2024-03-15,
3,6274509697,6224880200,Ata Start Semanal Stoic,Realizar reunião com Supera e José,Concluído,2024-03-20,2024-03-22
4,6274510080,6224880200,Ata Start Semanal Stoic,Compartilhar cronograma do BP,Vazio,2024-03-22,
...,...,...,...,...,...,...,...
3558,3961319299,3951153050,Finalizados - Ata Start Semanal,- Validar internamente relatório final;,Concluído,2023-02-17,
3559,3961319356,3951153050,Finalizados - Ata Start Semanal,- Enviar entregáveis finais para os clientes;,Concluído,2023-02-17,
3560,3961319396,3951153050,Finalizados - Ata Start Semanal,- Enviar NPS para os clientes.,Concluído,2023-02-17,
3561,4006289793,3951153050,Finalizados - Ata Start Semanal,- Tocar Reunião Interna de Transição do Projeto,Não Concluído,2023-02-17,


### Merge

In [ ]:
join_subelementos_elementos = (
    df_subelementos_atas[['id_elemento_dono']]
    .drop_duplicates()
    .merge(
        df_elementos_atas, 
        how='left', 
        left_on='id_elemento_dono', 
        right_on='id')
    [['id_elemento_dono', 'id_item_projeto']]
)

In [ ]:
df_subelementos_atas = df_subelementos_atas.merge(
    join_subelementos_elementos, 
    how='left', 
    left_on='id_elemento_dono', 
    right_on='id_elemento_dono'
)

In [ ]:
# REORDENAR COLUNAS

df_subelementos_atas = df_subelementos_atas[
    ['id',
    'id_elemento_dono',
    'id_item_projeto',
    'quadro_origem_pda',
    'atividade_pda',
    'status_pda',
    'data_planejada',
    'data_realizada'
    ]
]

In [ ]:
#df_subelementos_atas

## Boards Execução

### Gerar lista de boards

In [ ]:
metadados_boards_execucao = (
    todos_os_boards
    .loc[todos_os_boards['workspace_id'].isin(['2843713', '2405023'])]
    .loc[todos_os_boards['board_folder_id'].notnull()]
)

In [ ]:
lista_boards_execucao = metadados_boards_execucao['id'].astype('int64').values.tolist()

### Extração

In [ ]:
# importa os metadados de TODAS as colunas dos boards de Execução de cada projeto
#extrair_metadados_colunas(5950003005)

In [ ]:
# Selecionar só as colunas que interessam atualmente e definir nome estático pra cada uma

colunas_boards_execucao = {
    'person': 'equipe',
    'status': 'status_atividade',
    'date4': 'data_planejado',
    'numbers': 'esforco_previsto_horas',
    'date_1': 'data_realizado',
    'time_tracking': 'time_tracking',
    'data': 'data_criacao',
    'conectar_quadros': 'id_item_projeto',
    'item_name': 'atividade_execucao', # dado do item
    'item_group_title': 'etapa_projeto', # dado do board
    'duracao_total_segundos': 'esforco_utilizado_segundos'
}

    #'texto': 'etapa_coluna',
    #'texto6': 'etapa_coluna',

In [ ]:
agora = (pd.Timestamp.today() - pd.Timedelta(hours=3)).strftime("%Y-%m-%d %r")
counter = 0

dfs_atividades_projetos = []
time_alocado_execucao = []

print(f"Começando extração agora: {agora}")

for board in lista_boards_execucao:
    #print(f"EXTRAINDO BOARD {counter}\n")
    counter += 1

    dados_crus = (loop_extracao_grupos_v2(
        board, 
        colunas_boards_execucao, 
        print_an=False,
        ctrl_ext=False
        )
    )
    
    dict_dados_board = board_json_pra_dict(dados_crus, 'item_execucao')

    df_atividades = pd.DataFrame.from_dict(dict_dados_board['dados_achatados_board'])
    dfs_atividades_projetos.append(df_atividades)

    for item in dict_dados_board['time_alocado']:
        time_alocado_execucao.append(item)

    #df_alocacoes_execucao = pd.DataFrame.from_dict(dict_dados_board['time_alocado'])
    #dfs_alocacoes_execucao.append(df_alocacoes_execucao)

    if counter % 10 == 0:
        remaining = len(lista_boards_execucao) - counter
        print(f"Extraindo board número {counter}. Faltam {remaining}.")

Começando extração agora: 2024-06-10 01:24:39 PM
Extraindo board número 10. Faltam 267.
Extraindo board número 20. Faltam 257.
Extraindo board número 30. Faltam 247.
Extraindo board número 40. Faltam 237.
Extraindo board número 50. Faltam 227.
Extraindo board número 60. Faltam 217.
Extraindo board número 70. Faltam 207.
Extraindo board número 80. Faltam 197.
Extraindo board número 90. Faltam 187.
Extraindo board número 100. Faltam 177.
Extraindo board número 110. Faltam 167.
Extraindo board número 120. Faltam 157.
Extraindo board número 130. Faltam 147.
Extraindo board número 140. Faltam 137.
Extraindo board número 150. Faltam 127.
Extraindo board número 160. Faltam 117.
Extraindo board número 170. Faltam 107.
Extraindo board número 180. Faltam 97.
Extraindo board número 190. Faltam 87.
Extraindo board número 200. Faltam 77.
Extraindo board número 210. Faltam 67.
Extraindo board número 220. Faltam 57.
Extraindo board número 230. Faltam 47.
Extraindo board número 240. Faltam 37.
Extrain

In [ ]:
#time_alocado_execucao

In [ ]:
#pd.concat(dfs_atividades_projetos).rename(columns=colunas_boards_execucao)

### Tratamento

In [ ]:
df_boards_execucao = (
    pd.concat(dfs_atividades_projetos)
    .rename(columns=colunas_boards_execucao)
)

In [ ]:
#df_boards_execucao

In [ ]:
#df_boards_execucao#[['id', 'esforco_utilizado_segundos']].query("esforco_utilizado_segundos == 0")

In [ ]:
fp_time_alocado_execucao = f"{GDRIVE_BRUTOS}{hoje}_time_alocado_execucao.json"

with open(fp_time_alocado_execucao, "w") as outfile: 
    json.dump(time_alocado_execucao, outfile)

In [ ]:
aaadsddfg = '''
with open(fp_time_alocado_execucao, "r") as outfile: 
    adsd = json.load(outfile) '''

In [ ]:
filepath_execucao_naotratado = f"{GDRIVE_BRUTOS}{hoje}_dim_boards_execucao_naotratado.csv"
df_boards_execucao.to_csv(path_or_buf=filepath_execucao_naotratado, sep=';', index=False)

In [ ]:
# REORDENAR COLUNAS
df_boards_execucao = df_boards_execucao[[
    'id',
    'id_item_projeto',
    'id_quadro',
    'quadro_origem',
    'etapa_projeto',
    'atividade_execucao',
    'status_atividade',
    'data_criacao',
    'data_planejado',
    'data_realizado',
    'esforco_previsto_horas',
    'esforco_utilizado_segundos'
]]

In [ ]:
# DEFINIR QUAIS COLUNAS PRECISAM SER TRATADAS DE QUE FORMA

colunas_data_boards_execucao = [
    #'data_extracao',
    'data_criacao',
    'data_planejado',
    'data_realizado'
]

colunas_float_boards_execucao = [
    'esforco_previsto_horas',
    'esforco_utilizado_segundos'
]


# TRANSFORMAR AS COLUNAS NOS SEUS RESPECTIVOS FORMATOS

objeto_pra_datetime(df_boards_execucao, colunas_data_boards_execucao)
objeto_pra_float(df_boards_execucao, colunas_float_boards_execucao)

In [ ]:
# CRIAR COLUNAS CALCULADAS

df_boards_execucao['esforco_previsto_segundos'] = (
    (df_boards_execucao['esforco_previsto_horas'] * 3600)
    .fillna(0)
)

df_boards_execucao['deficit_esforco_segundos'] = (
    (df_boards_execucao['esforco_previsto_segundos'] - df_boards_execucao['esforco_utilizado_segundos'])
    .fillna(0)
)

In [ ]:
# DEIXAR TUDO COMO STRING OU OBJETO PRA SUBIR NO SHEETS

datetime_pra_date(df_boards_execucao, colunas_data_boards_execucao)
date_pra_string(df_boards_execucao, colunas_data_boards_execucao)
float_pra_string(df_boards_execucao, colunas_float_boards_execucao)



# APAGAR A COLUNA DE ESFORÇO PREVISTO EM HORAS PQ NÃO É RELEVANTE

df_boards_execucao = df_boards_execucao.drop(columns={'esforco_previsto_horas'})



# PREENCHER TODOS OS VAZIOS COM "" PRA SUBIR NO SHEETS
df_boards_execucao = df_boards_execucao.fillna("")

In [ ]:
df_boards_execucao.head()

,id,id_item_projeto,id_quadro,quadro_origem,etapa_projeto,atividade_execucao,status_atividade,data_criacao,data_planejado,data_realizado,esforco_utilizado_segundos,esforco_previsto_segundos,deficit_esforco_segundos
0,6699630361,,6699629935,427-2 FLAP (Estruturação Societária),Pré Operação,Preparação da Ata do Kick Off com Roteiro de D...,Done,,2024-05-23,2024-05-27,"0,0",1188.0,1188.0
1,6699630497,,6699629935,427-2 FLAP (Estruturação Societária),Pré Operação,Kick Off,Done,,2024-05-23,2024-05-23,"7200,0",3600.0,-3600.0
2,6699630538,,6699629935,427-2 FLAP (Estruturação Societária),Pré Operação,Elaboração Cronograma,Done,,2024-05-24,2024-05-24,"3240,0",1800.0,-1440.0
3,6699630601,,6699629935,427-2 FLAP (Estruturação Societária),Pré Operação,Elaboração Visão do Projeto,,,2024-05-27,,"0,0",3600.0,3600.0
4,6699630599,,6699629935,427-2 FLAP (Estruturação Societária),Análise Retroativa,Organização do Banco de Dados,Done,,2024-05-29,2024-06-03,"12857,0",14400.0,1543.0


## Projetos

### Extração

In [ ]:
# importa os metadados de TODAS as colunas do board de Gerenciamento

#extrair_metadados_colunas(3607290028)

In [ ]:
# Selecionar só as colunas que interessam atualmente e definir nome estático pra cada uma

colunas_gerenciamento = {
    'pessoas9': 'equipe',
    'sinal_de_confirma__o': 'criar_grupo',
    'label9': 'escopo_micro',
    'numeric': 'dificuldade_projeto',
    'n_meros9': 'prazo_contrato',
    'cronograma': 'cronograma_previsto',
    'data': 'data_inicio',
    'dup__of_data_de_in_cio': 'data_finalizacao',
    'espelho6': 'status_cronograma',
    'n_meros95': 'valor_contrato',
    'f_rmula1': 'parcelas_competencia',
    'formula': 'valor_parcela',
    'conectar_quadros5': 'id_item_funil',
    'lookup4': 'status_ata',
    'board_relation4': 'id_item_ata',
    'status': 'status_execucao',
    'boolean': 'nps_coletado',
    'avalia__o': 'nota_nps',
    'n_meros2': 'ad_projeto',
    'status_1': 'gatilho_pagamento',
    'item_name': 'nome_projeto', # dado do item
    'item_group_title': 'estagio_macro', # dado do board
    'cronograma_from': 'inicio_previsto', # esse e o próximo são da coluna de timerange
    'cronograma_to': 'final_previsto' 
}

In [ ]:
dados_brutos_projetos = loop_extracao_grupos_v2(3607290028, colunas_gerenciamento)

Começando extração de 4 grupos.

Extraindo grupo 1/4
Extraindo grupo 2/4
Extraindo grupo 3/4
Extraindo grupo 4/4

Finalizado.


In [ ]:
dict_projetos = board_json_pra_dict(dados_brutos_projetos, 'elemento_gerenciamento') 

In [ ]:
#pd.DataFrame.from_dict(dict_projetos['dados_achatados_board'])

### Extração e tratamento da coluna "Guardião"

Preferi fazer essa gambiarra de puxar esses dados separadamente pra não ter que alterar todo o método que faz o tratamento das colunas do tipo multiple-person

In [ ]:
dados_coluna_guardiao = loop_extracao_grupos_v2(3607290028, ['pessoas0'])

Começando extração de 4 grupos.

Extraindo grupo 1/4
Extraindo grupo 2/4
Extraindo grupo 3/4
Extraindo grupo 4/4

Finalizado.


In [ ]:
#dados_coluna_guardiao[1]#['items_page']['items']

In [ ]:
lista_coluna_guardiao = []

for group in dados_coluna_guardiao:
    for item in group['items_page']['items']:
        dict_guardiao = {
            'id': item['id'],
            'guardiao': item['column_values'][0]['text']
        }
        lista_coluna_guardiao.append(dict_guardiao)

In [ ]:
coluna_guardiao = pd.DataFrame.from_dict(lista_coluna_guardiao)

In [ ]:
#coluna_guardiao

### Tratamento

In [ ]:
# CRIA O DF E RENOMEIA COLUNAS DE ACORDO COM OS DICIONÁRIOS

df_projetos = (
    pd.DataFrame.from_dict(dict_projetos['dados_achatados_board'])
    .rename(columns=colunas_gerenciamento)
    .replace(r'^\s*$', np.nan, regex=True)
)

In [ ]:
#df_projetos

In [ ]:
# DEFINIR QUAIS COLUNAS PRECISAM SER TRATADAS DE QUE FORMA

colunas_data_projetos = [
    'data_extracao',
    'data_inicio',
    'data_finalizacao',
    'inicio_previsto',
    'final_previsto'
    ]

colunas_float_projetos = [
    'valor_contrato',
    'ad_projeto'
    ]


# TRANSFORMAR AS COLUNAS NOS SEUS RESPECTIVOS FORMATOS

objeto_pra_datetime(df_projetos, colunas_data_projetos)
objeto_pra_float(df_projetos, colunas_float_projetos)

In [ ]:
#df_projetos.info()

In [ ]:
# PREENCHER COLUNAS DE STATUS PRA MELHORAR A VISUALIZAÇÃO

df_projetos['status_cronograma'] = df_projetos['status_cronograma'].fillna('Vazio')

df_projetos['status_ata'] = df_projetos['status_ata'].fillna('Vazio')



# MUDAR COLUNAS BOOLEAN PRA TRUE EM VEZ DAQUELE CHECK

df_projetos['criar_grupo'] = df_projetos['criar_grupo'].replace('v', 'True')

df_projetos['nps_coletado'] = df_projetos['nps_coletado'].replace('v', 'True')

df_projetos['gatilho_pagamento'] = df_projetos['gatilho_pagamento'].replace('Sim', 'True')



# CRIAR COLUNAS CALCULADAS E ADICIONAR SEUS NOMES NO DICIONÁRIO DE COLUNAS FLOAT

df_projetos['dias_previstos'] = (df_projetos['final_previsto'] - df_projetos['inicio_previsto']).dt.days
colunas_float_projetos.append('dias_previstos')

df_projetos['valor_diario'] = (df_projetos['valor_contrato'] / df_projetos['dias_previstos']).round(2)
colunas_float_projetos.append('valor_diario')

df_projetos['parcelas_competencia'] = (df_projetos['dias_previstos'] / 30).round(0)
colunas_float_projetos.append('parcelas_competencia')

df_projetos['valor_parcela'] = df_projetos['valor_contrato'] / df_projetos['parcelas_competencia']
colunas_float_projetos.append('valor_parcela')

df_projetos['escopo_macro'] = np.where(
    df_projetos['escopo_micro'] == "M&A", "M&A", "Consultoria"
)

In [ ]:
# CRIA UMA SUBQUERY PRA PUXAR O NOME DE CADA QUADRO DE EXECUÇÃO

join_boards_projetos = (
    df_boards_execucao[['id_item_projeto', 'quadro_origem']]
    .rename(columns={'quadro_origem': 'quadro_projeto'})
    .drop_duplicates()
    .query("id_item_projeto != ''")
)

In [ ]:
# MERGE COM A SUBQUERY PRA PEGAR O NOME DO QUADRO
# Criei essa v2 no notebook antigo pra puxar mais colunas que são necessárias pra algumas vis novas
# Mas mantenho também a versão antiga pra não quebrar algumas vis já feitas

df_projetos_v2 = (
    df_projetos
    .merge(
        join_boards_projetos, 
        how='left', 
        left_on='id', 
        right_on='id_item_projeto')
    .merge(
        coluna_guardiao, 
        how='left', 
        left_on='id', 
        right_on='id')
    .drop(columns='id_item_projeto')
)

In [ ]:
# DEIXAR TUDO COMO STRING OU OBJETO PRA SUBIR NO SHEETS

datetime_pra_date(df_projetos_v2, colunas_data_projetos)
float_pra_string(df_projetos_v2, colunas_float_projetos)
date_pra_string(df_projetos_v2, colunas_data_projetos)

df_projetos_v2 = df_projetos_v2.fillna('')

In [ ]:
# A pedido da equipe da Stoic, excluí os projetos em fase de fechamento e os internos também
# Achei mais fácil alterar por aqui do que no Looker Studio

df_projetos_v2 = (df_projetos_v2.query(
    "estagio_macro != 'Projetos em Fase de Fechamento' and escopo_micro != 'Projeto Interno'")
)

In [ ]:
# SELECIONAR COLUNAS RELEVANTES PRA V1

df_projetos = df_projetos_v2[[
    'id',
    'quadro_origem',
    'quadro_projeto',
    'nome_projeto',
    'estagio_macro',
    'escopo_micro',
    'escopo_macro',
    'dificuldade_projeto',
    'prazo_contrato',
    'valor_contrato',
    'ad_projeto',
    'status_cronograma',
    'inicio_previsto',
    'final_previsto',
    'dias_previstos',
    'valor_diario',
    'data_inicio',
    'data_finalizacao',
    'guardiao'
    ]]

In [ ]:
df_projetos_v2 = df_projetos_v2[['data_extracao',
    'id',
    'id_quadro',
    'id_item_funil',
    'id_item_ata',
    'quadro_origem',
    'quadro_projeto',
    'nome_projeto',
    'estagio_macro',
    'criar_grupo',
    'escopo_micro',
    'escopo_macro',
    'dificuldade_projeto',
    'prazo_contrato',
    'inicio_previsto',
    'final_previsto',
    'dias_previstos',
    'data_inicio',
    'data_finalizacao',
    'status_cronograma',
    'valor_contrato',
    'parcelas_competencia',
    'valor_parcela',
    'valor_diario',
    'status_ata',
    'status_execucao',
    'nps_coletado',
    'nota_nps',
    'ad_projeto',
    'gatilho_pagamento',
    'guardiao'
]]

In [ ]:
#df_projetos_v2.info()

## Usuários

In [ ]:
users_query = '''query { 
    users (limit: 50) { 
        id
        enabled
        created_at
        name
        email
        is_admin
    }
}'''

r_users = requests.get(url=monday_apiUrl, json={'query' : users_query}, headers=headers)

#r_users.text

raw_users_data = json.loads(r_users.text)['data']

#pprint.pprint(raw_mna_board_data['boards'][0], sort_dicts=False)

In [ ]:
df_users = pd.DataFrame.from_dict(raw_users_data['users'])

In [ ]:
#df_users

## Timetracking

## Alocações

In [ ]:
# JUNTAR TODAS AS LISTAS ACHATADAS DE ALOCAÇÃO

all_allocation = (
    dict_projetos['time_alocado'] + 
    dict_resposta_ata_ativos['time_alocado'] + 
    dict_resposta_ata_finalizados['time_alocado'] + 
    dict_subitems_atas_ativos['time_alocado'] + 
    dict_subitems_atas_finalizados['time_alocado'] + 
    time_alocado_execucao +
    dict_csat['time_alocado']
)

df_alocacoes = pd.DataFrame.from_dict(all_allocation)#.astype({'id_pessoa': 'object'})

In [ ]:
# JOIN PRA PEGAR O NOME DA PESSOA

df_alocacoes = (
    df_alocacoes
    .merge((df_users[['id', 'name']].astype({'id': 'int64'})), how='left', left_on='id_pessoa', right_on='id')
    .drop(columns='id')
)

In [ ]:
# REORDENAR COLUNAS

df_alocacoes = df_alocacoes[[
    'id_item',
    'quadro_origem',
    'granularidade',
    'id_pessoa',
    'name'
]]

In [ ]:
#df_alocacoes

# Views

## PDA's Diários

In [ ]:
view_pdas_diarios = (lambda: _deepnote_execute_sql('-- tá dando um pouco de diferença da API antiga porque não puxo aquelas subtasks que não pertencem a grupos\n\nWITH tab_subelementos AS (\n    SELECT\n        df_subelementos_atas.id,\n        df_subelementos_atas.data_planejada,\n        df_subelementos_atas.data_realizada,\n        df_projetos.nome_projeto,\n        df_projetos.estagio_macro,\n        df_projetos.escopo_micro\n    FROM df_subelementos_atas\n    LEFT JOIN df_projetos\n        ON df_subelementos_atas.id_item_projeto = df_projetos.id\n),\n\nplanejados AS (\n    SELECT\n        data_planejada,\n        nome_projeto,\n        estagio_macro,\n        escopo_micro,\n        COUNT(*) AS "pdas_planejados"\n    FROM tab_subelementos\n    GROUP BY 1, 2, 3, 4\n    ORDER BY 1 ASC, 2 ASC\n),\n\nrealizados AS (\n    SELECT\n        data_realizada,\n        nome_projeto,\n        estagio_macro,\n        escopo_micro,\n        COUNT(*) AS "pdas_realizados"\n    FROM tab_subelementos\n    GROUP BY 1, 2, 3, 4\n    ORDER BY 1 ASC, 2 ASC\n),\n\ntudo AS (\n    SELECT\n        CASE WHEN data_planejada IS NULL THEN data_realizada ELSE data_planejada END AS "data",\n        CASE WHEN planejados.nome_projeto IS NULL THEN realizados.nome_projeto ELSE planejados.nome_projeto END AS "nome_projeto",\n        CASE WHEN planejados.estagio_macro IS NULL THEN realizados.estagio_macro ELSE planejados.estagio_macro END AS "estagio_macro",\n        CASE WHEN planejados.escopo_micro IS NULL THEN realizados.escopo_micro ELSE planejados.escopo_micro END AS "escopo_micro",\n        CASE WHEN pdas_planejados IS NULL THEN 0 ELSE pdas_planejados END AS "pdas_planejados",\n        CASE WHEN pdas_realizados IS NULL THEN 0 ELSE pdas_realizados END AS "pdas_realizados",\n    FROM planejados\n    FULL OUTER JOIN realizados\n        ON planejados.data_planejada = realizados.data_realizada\n        AND planejados.nome_projeto = realizados.nome_projeto\n        AND planejados.estagio_macro = realizados.estagio_macro\n        AND planejados.escopo_micro = realizados.escopo_micro\n    ORDER BY 1 ASC, 2 ASC\n)\n\nSELECT * FROM tudo', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('-- tá dando um pouco de diferença da API antiga porque não puxo aquelas subtasks que não pertencem a grupos\n\nWITH tab_subelementos AS (\n    SELECT\n        df_subelementos_atas.id,\n        df_subelementos_atas.data_planejada,\n        df_subelementos_atas.data_realizada,\n        df_projetos.nome_projeto,\n        df_projetos.estagio_macro,\n        df_projetos.escopo_micro\n    FROM df_subelementos_atas\n    LEFT JOIN df_projetos\n        ON df_subelementos_atas.id_item_projeto = df_projetos.id\n),\n\nplanejados AS (\n    SELECT\n        data_planejada,\n        nome_projeto,\n        estagio_macro,\n        escopo_micro,\n        COUNT(*) AS "pdas_planejados"\n    FROM tab_subelementos\n    GROUP BY 1, 2, 3, 4\n    ORDER BY 1 ASC, 2 ASC\n),\n\nrealizados AS (\n    SELECT\n        data_realizada,\n        nome_projeto,\n        estagio_macro,\n        escopo_micro,\n        COUNT(*) AS "pdas_realizados"\n    FROM tab_subelementos\n    GROUP BY 1, 2, 3, 4\n    ORDER BY 1 ASC, 2 ASC\n),\n\ntudo AS (\n    SELECT\n        CASE WHEN data_planejada IS NULL THEN data_realizada ELSE data_planejada END AS "data",\n        CASE WHEN planejados.nome_projeto IS NULL THEN realizados.nome_projeto ELSE planejados.nome_projeto END AS "nome_projeto",\n        CASE WHEN planejados.estagio_macro IS NULL THEN realizados.estagio_macro ELSE planejados.estagio_macro END AS "estagio_macro",\n        CASE WHEN planejados.escopo_micro IS NULL THEN realizados.escopo_micro ELSE planejados.escopo_micro END AS "escopo_micro",\n        CASE WHEN pdas_planejados IS NULL THEN 0 ELSE pdas_planejados END AS "pdas_planejados",\n        CASE WHEN pdas_realizados IS NULL THEN 0 ELSE pdas_realizados END AS "pdas_realizados",\n    FROM planejados\n    FULL OUTER JOIN realizados\n        ON planejados.data_planejada = realizados.data_realizada\n        AND planejados.nome_projeto = realizados.nome_projeto\n        AND planejados.estagio_macro = realizados.estagio_macro\n        AND planejados.escopo_micro = realizados.escopo_micro\n    ORDER BY 1 ASC, 2 ASC\n)\n\nSELECT * FROM tudo', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
view_pdas_diarios

,data,nome_projeto,estagio_macro,escopo_micro,pdas_planejados,pdas_realizados
0,,022-1 Grupo Anhumas,Projetos Finalizados,M&A,0,17
1,,062-1 Brasfort,Projetos Finalizados,Viabilidade Econômica-Financeira,8,8
2,,062-2 Brasfort (Área Comercial),Projetos Finalizados,Advising,0,2
3,,081-1 OtorrinoDF <> Otorrino Gama,Projetos Finalizados,Valuation,0,3
4,,081-2 OtorrinoDF <> MedLago,Projetos Finalizados,Valuation,0,3
...,...,...,...,...,...,...
4236,2024-03-22,372-3 Ecosys M&A,Projetos Finalizados,M&A,0,1
4237,2024-03-22,402-1 Casa Park,Projetos Ativos,Crédito,2,1
4238,2024-03-23,295-12 Unifarma - VTC,Projetos Ativos,M&A,2,0
4239,2024-03-23,372-3 Ecosys M&A,Projetos Finalizados,M&A,1,0


In [ ]:
view_pdas_diarios['semana'] = (
    view_pdas_diarios['data']
    .replace(r'^\s*$', '2001-01-01', regex=True)
    .astype('datetime64[ns]')
    .dt.to_period('W')
    .apply(lambda r: r.start_time)
    .dt.date
    .astype('string')
)

view_pdas_diarios['semana'] = np.where(
    view_pdas_diarios['semana'] == "2001-01-01", 
    "", 
    view_pdas_diarios['semana']
)

In [ ]:
#view_pdas_diarios

## Alocações em projetos por mês

In [ ]:
view_alocacoes_mes = (lambda: _deepnote_execute_sql('WITH cte1 AS (\n    SELECT\n        df_boards_execucao.id,\n        df_boards_execucao.id_quadro,\n        df_boards_execucao.quadro_origem,\n        df_boards_execucao.data_realizado,\n        df_calendario.mes,\n        df_alocacoes.id_pessoa\n    FROM df_boards_execucao\n    LEFT JOIN df_calendario\n        ON df_boards_execucao.data_realizado = df_calendario.data\n    LEFT JOIN df_alocacoes\n        ON df_boards_execucao.id = df_alocacoes.id_item\n),\n\ncte2 AS (\n    SELECT\n        id_quadro,\n        quadro_origem,\n        mes,\n        id_pessoa,\n        COUNT(*) AS "atividades"\n    FROM cte1\n    WHERE id_pessoa IS NOT NULL AND mes IS NOT NULL\n    GROUP BY 1, 2, 3, 4\n    ORDER BY 1, 2, 3, 4\n),\n\ncte3 AS (\n    SELECT\n        mes,\n        id_pessoa,\n        COUNT(DISTINCT id_quadro) AS "projetos"\n    FROM cte2\n    GROUP BY 1, 2\n    ORDER BY 1, 2\n)\n\nSELECT\n    *\nFROM cte3', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('WITH cte1 AS (\n    SELECT\n        df_boards_execucao.id,\n        df_boards_execucao.id_quadro,\n        df_boards_execucao.quadro_origem,\n        df_boards_execucao.data_realizado,\n        df_calendario.mes,\n        df_alocacoes.id_pessoa\n    FROM df_boards_execucao\n    LEFT JOIN df_calendario\n        ON df_boards_execucao.data_realizado = df_calendario.data\n    LEFT JOIN df_alocacoes\n        ON df_boards_execucao.id = df_alocacoes.id_item\n),\n\ncte2 AS (\n    SELECT\n        id_quadro,\n        quadro_origem,\n        mes,\n        id_pessoa,\n        COUNT(*) AS "atividades"\n    FROM cte1\n    WHERE id_pessoa IS NOT NULL AND mes IS NOT NULL\n    GROUP BY 1, 2, 3, 4\n    ORDER BY 1, 2, 3, 4\n),\n\ncte3 AS (\n    SELECT\n        mes,\n        id_pessoa,\n        COUNT(DISTINCT id_quadro) AS "projetos"\n    FROM cte2\n    GROUP BY 1, 2\n    ORDER BY 1, 2\n)\n\nSELECT\n    *\nFROM cte3', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
view_alocacoes_mes

,mes,id_pessoa,projetos
0,2021-01-01,15963645,2
1,2021-02-01,15963645,2
2,2021-03-01,15872615,1
3,2021-03-01,15963645,2
4,2021-03-01,15966591,1
...,...,...,...
402,2024-06-01,16973544,3
403,2024-06-01,30529234,8
404,2024-06-01,57009654,4
405,2024-06-01,58041788,3


## Metas

In [ ]:
metas_necessarias = '''
FATURAMENTO COMPETÊNCIA
- soma mensal da coluna Valor da Parcela no gerenciamento
COMERCIAL
- soma mensal da coluna Valor no Funil comercial
PROPOSTAS
- funil comercial - data de proposta
CONTRATOS
- funil comercial - data de fechamento
CONVERSÃO
- coluna calculada
FIDELIZAÇÃO
- funil comercial, coluna Origem
TICKET MÉDIO
- coluna calculada. faturamento por competência / projetos ativos
NPS
- board de NPS
- trazer na conta de NPS mesmo
NPSs COLETADOS
- board NPS
% NPSs COLETADOS
- count de NPSs coletados / projetos finalizados no mês
PROJETOS FINALIZADOS
- gerenciamento
PROJETOS ATIVOS
- gerenciamento
PROJETOS FIN NO CRONOGRAMA
- gerenciamento. # de projetos com cronograma "Em dia"
PROJETOS ATIVOS NO CRONOGRAMA
- gerenciamento # de projetos com cronograma "Em dia"
% FIDELIZAÇÃO
- coluna calculada
AD PESSOAS
- board AD pessoal
AD PROJETOS
- board AD projetos
'''

## Faturamento por Competência

In [ ]:
view_faturamento_competencia = (lambda: _deepnote_execute_sql('WITH cte1 AS (\n    SELECT\n        id,\n        nome_projeto,\n        CASE\n            WHEN inicio_previsto = \'\' THEN NULL\n            ELSE DATE_TRUNC(\'month\', CAST(inicio_previsto AS DATE))\n            END AS "mes_inicio",\n        CASE\n            WHEN parcelas_competencia = \'\' THEN NULL\n            ELSE CAST(REPLACE(parcelas_competencia, \',\', \'.\') AS INT)\n            END AS parcelas_competencia,\n        CASE\n            WHEN valor_parcela = \'\' THEN NULL\n            WHEN valor_parcela = \'inf\' THEN NULL\n            ELSE CAST(REPLACE(valor_parcela, \',\', \'.\') AS INT)\n            END AS valor_parcela\n    FROM df_projetos_v2\n),\n\ncte2 as (\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n),\n\ncte3 AS (\n    SELECT\n        cte2.*,\n        ROW_NUMBER() OVER(PARTITION BY id ORDER BY id) AS "contador_mes"\n    FROM cte2\n),\n\ncte4 AS (\n    SELECT\n        cte3.*,\n        DATE_ADD(mes_inicio, INTERVAL (contador_mes - 1) MONTH) AS "mes_competencia"\n    FROM cte3\n    WHERE contador_mes <= parcelas_competencia\n)\n\nSELECT * FROM cte4', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('WITH cte1 AS (\n    SELECT\n        id,\n        nome_projeto,\n        CASE\n            WHEN inicio_previsto = \'\' THEN NULL\n            ELSE DATE_TRUNC(\'month\', CAST(inicio_previsto AS DATE))\n            END AS "mes_inicio",\n        CASE\n            WHEN parcelas_competencia = \'\' THEN NULL\n            ELSE CAST(REPLACE(parcelas_competencia, \',\', \'.\') AS INT)\n            END AS parcelas_competencia,\n        CASE\n            WHEN valor_parcela = \'\' THEN NULL\n            WHEN valor_parcela = \'inf\' THEN NULL\n            ELSE CAST(REPLACE(valor_parcela, \',\', \'.\') AS INT)\n            END AS valor_parcela\n    FROM df_projetos_v2\n),\n\ncte2 as (\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n    UNION ALL\n    SELECT * FROM cte1\n),\n\ncte3 AS (\n    SELECT\n        cte2.*,\n        ROW_NUMBER() OVER(PARTITION BY id ORDER BY id) AS "contador_mes"\n    FROM cte2\n),\n\ncte4 AS (\n    SELECT\n        cte3.*,\n        DATE_ADD(mes_inicio, INTERVAL (contador_mes - 1) MONTH) AS "mes_competencia"\n    FROM cte3\n    WHERE contador_mes <= parcelas_competencia\n)\n\nSELECT * FROM cte4', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
view_faturamento_competencia

,id,nome_projeto,mes_inicio,parcelas_competencia,valor_parcela,contador_mes,mes_competencia
0,4014156051,248-2 SCG - LOTEPAR,2023-03-01,3,36667.0,1,2023-03-01
1,4014156051,248-2 SCG - LOTEPAR,2023-03-01,3,36667.0,2,2023-04-01
2,4014156051,248-2 SCG - LOTEPAR,2023-03-01,3,36667.0,3,2023-05-01
3,4619393097,Fabrik - Alex Griebeler,2023-06-01,2,7500.0,1,2023-06-01
4,4619393097,Fabrik - Alex Griebeler,2023-06-01,2,7500.0,2,2023-07-01
...,...,...,...,...,...,...,...
540,4787833111,087-1 Diagnose M&A,2022-03-01,21,1429.0,11,2023-01-01
541,4787833111,087-1 Diagnose M&A,2022-03-01,21,1429.0,12,2023-02-01
542,4787833262,KOI,2023-03-01,3,NaN,1,2023-03-01
543,4787833262,KOI,2023-03-01,3,NaN,2,2023-04-01


In [ ]:
datetime_pra_date(view_faturamento_competencia, ['mes_inicio', 'mes_competencia'])
float_pra_string(view_faturamento_competencia, ['valor_parcela'])
date_pra_string(view_faturamento_competencia, ['mes_inicio', 'mes_competencia'])

In [ ]:
cols_view_faturamento = ['id', 'contador_mes', 'mes_competencia', 'valor_parcela']

view_faturamento_competencia = view_faturamento_competencia[cols_view_faturamento].fillna('')

In [ ]:
#view_faturamento_competencia

## Conversão do funil

In [ ]:
view_conversao_mes = (lambda: _deepnote_execute_sql('WITH cte1 AS (\n    SELECT\n        DATE_TRUNC(\'month\', CAST(data_proposta AS DATE)) AS "mes_proposta",\n        COUNT(*) AS "propostas_apresentadas"\n    FROM df_funil\n    WHERE data_proposta != \'\'\n    GROUP BY 1\n    ORDER BY 1 ASC\n),\n\ncte2 as (\n    SELECT\n        DATE_TRUNC(\'month\', CAST(data_proposta AS DATE)) AS "mes_proposta",\n        COUNT(*) AS "propostas_negadas"\n    FROM df_funil \n    WHERE data_proposta != \'\' AND estagio_negociacao = \'Leads Declinados\'\n    GROUP BY 1\n    ORDER BY 1\n),\n\ncte3 AS (\n    SELECT\n        DATE_TRUNC(\'month\', CAST(data_fechamento AS DATE)) AS "mes_fechamento",\n        COUNT(*) AS "projetos_fechados"\n    FROM df_funil\n    WHERE data_fechamento != \'\'\n    GROUP BY 1\n    ORDER BY 1 ASC\n),\n\ncte4 AS (\n    SELECT\n        cte1.*,\n        CASE WHEN cte2.propostas_negadas IS NULL THEN 0 ELSE propostas_negadas END AS "propostas_negadas",\n        CASE WHEN cte3.projetos_fechados IS NULL THEN 0 ELSE projetos_fechados END AS "projetos_fechados"\n    FROM cte1\n    LEFT JOIN cte2 ON cte1.mes_proposta = cte2.mes_proposta\n    LEFT JOIN cte3 ON cte1.mes_proposta = cte3.mes_fechamento\n    ORDER BY 1 ASC\n),\n\ncte5 AS (\n    SELECT\n        cte4.*,\n        propostas_negadas + projetos_fechados AS "respostas_definitivas",\n        ROUND(projetos_fechados / (projetos_fechados + propostas_negadas), 2) AS "conversao"\n    FROM cte4\n)\n\nSELECT\n    *\nFROM cte5', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('WITH cte1 AS (\n    SELECT\n        DATE_TRUNC(\'month\', CAST(data_proposta AS DATE)) AS "mes_proposta",\n        COUNT(*) AS "propostas_apresentadas"\n    FROM df_funil\n    WHERE data_proposta != \'\'\n    GROUP BY 1\n    ORDER BY 1 ASC\n),\n\ncte2 as (\n    SELECT\n        DATE_TRUNC(\'month\', CAST(data_proposta AS DATE)) AS "mes_proposta",\n        COUNT(*) AS "propostas_negadas"\n    FROM df_funil \n    WHERE data_proposta != \'\' AND estagio_negociacao = \'Leads Declinados\'\n    GROUP BY 1\n    ORDER BY 1\n),\n\ncte3 AS (\n    SELECT\n        DATE_TRUNC(\'month\', CAST(data_fechamento AS DATE)) AS "mes_fechamento",\n        COUNT(*) AS "projetos_fechados"\n    FROM df_funil\n    WHERE data_fechamento != \'\'\n    GROUP BY 1\n    ORDER BY 1 ASC\n),\n\ncte4 AS (\n    SELECT\n        cte1.*,\n        CASE WHEN cte2.propostas_negadas IS NULL THEN 0 ELSE propostas_negadas END AS "propostas_negadas",\n        CASE WHEN cte3.projetos_fechados IS NULL THEN 0 ELSE projetos_fechados END AS "projetos_fechados"\n    FROM cte1\n    LEFT JOIN cte2 ON cte1.mes_proposta = cte2.mes_proposta\n    LEFT JOIN cte3 ON cte1.mes_proposta = cte3.mes_fechamento\n    ORDER BY 1 ASC\n),\n\ncte5 AS (\n    SELECT\n        cte4.*,\n        propostas_negadas + projetos_fechados AS "respostas_definitivas",\n        ROUND(projetos_fechados / (projetos_fechados + propostas_negadas), 2) AS "conversao"\n    FROM cte4\n)\n\nSELECT\n    *\nFROM cte5', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
view_conversao_mes

,mes_proposta,propostas_apresentadas,propostas_negadas,projetos_fechados,respostas_definitivas,conversao
0,2020-01-01,1,0,0,0,NaN
1,2022-03-01,12,0,7,7,1.00
2,2022-04-01,5,0,7,7,1.00
3,2022-05-01,4,0,3,3,1.00
4,2022-08-01,1,0,4,4,1.00
5,2022-09-01,1,0,2,2,1.00
6,2022-10-01,1,0,8,8,1.00
7,2022-11-01,1,0,7,7,1.00
8,2022-12-01,2,0,2,2,1.00
9,2023-01-01,4,0,4,4,1.00


In [ ]:
date_pra_string(view_conversao_mes, ['mes_proposta'])

# Salvamentos

## Atualização Sheets

In [ ]:
STOIC_MONDAY_SHEETS = '1qaXV_SGSTuZd9DyXqJ4h1aTAc4E4EtF4Zni8qWfvteA'
STOIC_VIEWS_SHEETS = '1pZXbth01PaK0xxOvlNMRxfm-eI4dNBquz_2EhLUnyWc'

RANGES = {
    'teste': 'teste_sheets!A2:O',
    'df_projetos': 'projetos!A2:S',
    'df_projetos_v2': 'projetosv2!A2:AE',
    'df_elementos': 'elementos!A2:I',
    'df_subelementos': 'subelementos!A2:H',
    'df_alocacoes': 'equipe_alocada!A2:E',
    #'df_nps': 'nps!A2:E',      # board apagado
    'df_csat': 'csat!A2:S',
    'df_boards_execucao': 'boards_execucao!A2:M',
    'df_funil': 'funil!A2:W',
    'df_ad_pessoal': 'ad_pessoal!A2:R',
    'df_ad_projeto': 'ad_projeto!A2:AG',
    #'sql_view_nps_projeto': 'nps_projeto!A2:I',
    #'sql_view_nps_pessoa': 'nps_pessoa!A2:L',
    'view_pdas_diarios': 'pdas_diarios!A2:G',
    'view_alocacoes_mes': 'alocacoes_mes!A2:C',
    'view_faturamento_competencia': 'faturamento_competencia!A2:E',
    'view_conversao_mes': 'conversao_mes!A2:F'
}

In [ ]:
def update_sheet(df, sheet, sheet_range):
    agora = (pd.Timestamp.today() - pd.Timedelta(hours=3)).strftime("%Y-%m-%d %r")
    print(f"Updating range: {sheet_range}")
    
    # variáveis gerais
    df = df.fillna(value='')
    value_list = []
    service = build('sheets', 'v4', credentials=creds)

    # limpar a planilha
    bodyClear = {}
    resultClear = service.spreadsheets().values().clear(
        spreadsheetId=sheet,
        range=sheet_range,
        body=bodyClear).execute()
    print("Cells cleared.")

    for index, rows in df.iterrows():
        row_list = rows.values.flatten().tolist()
        value_list.append(row_list)

    # preencher os valores de fato
    values = value_list
    body = {
        'values': values
    }
    result = service.spreadsheets().values().update(
        spreadsheetId=sheet,
        range=sheet_range,
        valueInputOption='USER_ENTERED',
        body=body).execute()

    print(f"{result.get('updatedCells')} cells updated.")
    print(f"Sheet updated at {agora}")
    print("")

In [ ]:
def update_all_sheets():
    update_sheet(df_projetos, STOIC_MONDAY_SHEETS, RANGES['df_projetos'])
    update_sheet(df_projetos_v2, STOIC_MONDAY_SHEETS, RANGES['df_projetos_v2'])
    update_sheet(df_elementos_atas, STOIC_MONDAY_SHEETS, RANGES['df_elementos'])
    update_sheet(df_subelementos_atas, STOIC_MONDAY_SHEETS, RANGES['df_subelementos'])
    update_sheet(df_alocacoes, STOIC_MONDAY_SHEETS, RANGES['df_alocacoes'])
    #update_sheet(df_nps, STOIC_MONDAY_SHEETS, RANGES['df_nps'])
    update_sheet(df_csat, STOIC_MONDAY_SHEETS, RANGES['df_csat'])
    update_sheet(df_boards_execucao, STOIC_MONDAY_SHEETS, RANGES['df_boards_execucao'])
    update_sheet(df_funil, STOIC_MONDAY_SHEETS, RANGES['df_funil'])
    update_sheet(df_ad_pessoal, STOIC_MONDAY_SHEETS, RANGES['df_ad_pessoal'])
    update_sheet(df_ad_projeto, STOIC_MONDAY_SHEETS, RANGES['df_ad_projeto'])
    #update_sheet(sql_view_nps_projeto, STOIC_VIEWS_SHEETS, RANGES['sql_view_nps_projeto'])
    #update_sheet(sql_view_nps_pessoa, STOIC_VIEWS_SHEETS, RANGES['sql_view_nps_pessoa'])
    update_sheet(view_pdas_diarios, STOIC_VIEWS_SHEETS, RANGES['view_pdas_diarios'])
    update_sheet(view_alocacoes_mes, STOIC_VIEWS_SHEETS, RANGES['view_alocacoes_mes'])
    update_sheet(view_faturamento_competencia, STOIC_VIEWS_SHEETS, RANGES['view_faturamento_competencia'])
    update_sheet(view_conversao_mes, STOIC_VIEWS_SHEETS, RANGES['view_conversao_mes'])

In [ ]:
update_all_sheets()

Updating range: projetos!A2:S
Cells cleared.
2717 cells updated.
Sheet updated at 2024-06-10 01:51:28 PM

Updating range: projetosv2!A2:AE
Cells cleared.
4433 cells updated.
Sheet updated at 2024-06-10 01:51:31 PM

Updating range: elementos!A2:I
Cells cleared.
5013 cells updated.
Sheet updated at 2024-06-10 01:51:32 PM

Updating range: subelementos!A2:H
Cells cleared.
39584 cells updated.
Sheet updated at 2024-06-10 01:51:33 PM

Updating range: equipe_alocada!A2:E
Cells cleared.
142790 cells updated.
Sheet updated at 2024-06-10 01:51:36 PM

Updating range: csat!A2:S
Cells cleared.
1596 cells updated.
Sheet updated at 2024-06-10 01:51:41 PM

Updating range: boards_execucao!A2:M
Cells cleared.
229879 cells updated.
Sheet updated at 2024-06-10 01:51:42 PM

Updating range: funil!A2:W
Cells cleared.
14398 cells updated.
Sheet updated at 2024-06-10 01:51:50 PM

Updating range: ad_pessoal!A2:R
Cells cleared.
1224 cells updated.
Sheet updated at 2024-06-10 01:51:51 PM

Updating range: ad_proje

## Gravação Drive

In [ ]:
filepath_projects = f"{GDRIVE_APINOVA}{hoje}_dim_projetos.csv"
filepath_projects_v2 = f"{GDRIVE_APINOVA}{hoje}_dim_projetos_v2.csv"
filepath_elements = f"{GDRIVE_APINOVA}{hoje}_dim_elementos_atas.csv"
filepath_subelements = f"{GDRIVE_APINOVA}{hoje}_dim_subelementos_atas.csv"
filepath_users = f"{GDRIVE_APINOVA}{hoje}_dim_usuarios.csv"
filepath_team_allocation = f"{GDRIVE_APINOVA}{hoje}_dim_equipe_alocada.csv"
#filepath_nps = f"{GDRIVE_APINOVA}{hoje}_dim_nps.csv"
filepath_csat = f"{GDRIVE_APINOVA}{hoje}_dim_csat.csv"
filepath_execution_boards = f"{GDRIVE_APINOVA}{hoje}_dim_boards_execucao.csv"
filepath_funil = f"{GDRIVE_APINOVA}{hoje}_dim_funil_comercial.csv"
filepath_ad_pessoal = f"{GDRIVE_APINOVA}{hoje}_dim_ad_pessoal.csv"
filepath_ad_projeto = f"{GDRIVE_APINOVA}{hoje}_dim_ad_projeto.csv"

#fp_view_nps_projeto = f"{GDRIVE_APINOVA}{hoje}_view_nps_projeto.csv"
#fp_view_nps_pessoa = f"{GDRIVE_APINOVA}{hoje}_view_nps_pessoa.csv"
fp_view_alocacoes_mes = f"{GDRIVE_APINOVA}{hoje}_view_alocacoes_mes.csv"
fp_view_pdas_diarios = f"{GDRIVE_APINOVA}{hoje}_view_pdas_diarios.csv"
fp_view_fat_competencia = f"{GDRIVE_APINOVA}{hoje}_view_fat_competencia.csv"
fp_view_conversao_mes = f"{GDRIVE_APINOVA}{hoje}_view_conversao_mes.csv"

In [ ]:
def gravar_tudo_drive():
    agora = (pd.Timestamp.today() - pd.Timedelta(hours=3)).strftime("%Y-%m-%d %r")
    print(f"Iniciando gravações no Drive.")
    print("")

    print("Gravando: df_projetos")
    df_projetos.to_csv(path_or_buf=filepath_projects, sep=';', index=False)
    print("Gravando: df_projetos_v2")
    df_projetos_v2.to_csv(path_or_buf=filepath_projects_v2, sep=';', index=False)
    print("Gravando: df_elementos_atas")
    df_elementos_atas.to_csv(path_or_buf=filepath_elements, sep=';', index=False)
    print("Gravando: df_subelementos_atas")
    df_subelementos_atas.to_csv(path_or_buf=filepath_subelements, sep=';', index=False)
    print("Gravando: df_users")
    df_users.to_csv(path_or_buf=filepath_users, sep=';', index=False)
    print("Gravando: df_alocacoes")
    df_alocacoes.to_csv(path_or_buf=filepath_team_allocation, sep=';', index=False)
    #print("Gravando: df_nps")
    #df_nps.to_csv(path_or_buf=filepath_nps, sep=';', index=False)
    print("Gravando: df_csat")
    df_csat.to_csv(path_or_buf=filepath_csat, sep=';', index=False)
    print("Gravando: df_boards_execucao")
    df_boards_execucao.to_csv(path_or_buf=filepath_execution_boards, sep=';', index=False)
    print("Gravando: df_ad_pessoal")
    df_ad_pessoal.to_csv(path_or_buf=filepath_ad_pessoal, sep=';', index=False)
    print("Gravando: df_ad_projeto")
    df_ad_projeto.to_csv(path_or_buf=filepath_ad_projeto, sep=';', index=False)
    print("Gravando: df_funil")
    df_funil.to_csv(path_or_buf=filepath_funil, sep=';', index=False)
    print("")

    #print("Gravando: nps_projeto")
    #sql_view_nps_projeto.to_csv(path_or_buf=fp_view_nps_projeto, sep=';', index=False)
    #print("Gravando: nps_pessoa")
    #sql_view_nps_pessoa.to_csv(path_or_buf=fp_view_nps_pessoa, sep=';', index=False)
    print("Gravando: alocacoes_mes")
    view_alocacoes_mes.to_csv(path_or_buf=fp_view_alocacoes_mes, sep=';', index=False)
    print("Gravando: pdas_diarios")
    view_pdas_diarios.to_csv(path_or_buf=fp_view_pdas_diarios, sep=';', index=False)
    print("Gravando: faturamento_competencia")
    view_faturamento_competencia.to_csv(path_or_buf=fp_view_fat_competencia, sep=';', index=False)
    print("Gravando: conversao_mes")
    view_conversao_mes.to_csv(path_or_buf=fp_view_conversao_mes, sep=';', index=False)

    print("")
    print(f"Terminado agora: {agora}")

In [ ]:
gravar_tudo_drive()

Iniciando gravações no Drive.

Gravando: df_projetos
Gravando: df_projetos_v2
Gravando: df_elementos_atas
Gravando: df_subelementos_atas
Gravando: df_users
Gravando: df_alocacoes
Gravando: df_csat
Gravando: df_boards_execucao
Gravando: df_ad_pessoal
Gravando: df_ad_projeto
Gravando: df_funil

Gravando: alocacoes_mes
Gravando: pdas_diarios
Gravando: faturamento_competencia
Gravando: conversao_mes

Terminado agora: 2024-06-10 01:52:06 PM


# Testes

In [ ]:
### df_projetos
### df_projetos_v2
### df_elementos_atas
### df_subelementos_atas
### df_alocacoes
### df_csat
### df_boards_execucao
### df_funil
### df_ad_pessoal
### df_ad_projeto
### view_pdas_diarios
### view_alocacoes_mes
### view_faturamento_competencia
### view_conversao_mes

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=965628b0-455f-4543-bebf-a9bfbf07ce66' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>